In [20]:
import os
import re
import math
import logging
import random
import functools

import sqlite3
import json
import pandas as pd
import numpy as np
import itertools as tt

#from collections import Counter
import matplotlib.pyplot as plt

logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', datefmt='%Y-%m-%d %I:%M:%S')
logging.getLogger().setLevel(logging.INFO)
logging.info("Logging set.")

2023-01-24 09:38:07 INFO: Logging set.


In [21]:
def get_integer_input(filename):
    retval = None
    with open(filename) as IN:
        retval = [int(x) for x in IN.read().splitlines()]
    logging.info("Got {} integers from {}.".format(len(retval),filename))
    return retval

def get_string_input(filename):
    retval = None
    with open(filename) as IN:
        retval = IN.read().splitlines()
    logging.info("Got {} lines from {}.".format(len(retval),filename))
    return retval

def reverse_graph(A):
    ''' Return the transpose of the adjency matrix A
            A is a dict of dicts;  A[x][y] = value
            retval[x][y] = A[y][x]
    '''
    retval = dict()
    for row in A.keys():
        retval[row] = dict()
    for row, cols in A.items():
        for col in cols.keys():
            retval[col][row] = A[row][col]
    return retval

def l1_norm(vec):
    return sum([abs(v) for v in vec])

def get_grid_index(r, c, n, m):
    return r*m + c


def get_grid_neighborhood(ind, n, m, diagonal=True):
    ''' Return the indices that are adjacent to [r,c] in a grid of size nxm. '''
    retval = []
    size = n*m
    aboveRow = [ind-m-1, ind-m, ind-m+1]
    belowRow = [ind+m-1, ind+m, ind+m+1]
    if diagonal == False:
        aboveRow = [ind-m]
        belowRow = [ind+m]
        
    for newInd in aboveRow:
        if 0 <= newInd and newInd < size and ind//m == (newInd//m) + 1:
            retval.append(newInd)
    for newInd in [ind-1, ind+1]:
        if 0 <= newInd and newInd < size and ind//m == newInd//m:
            retval.append(newInd)
    for newInd in belowRow:
        if 0 <= newInd and newInd < size and ind//m == (newInd//m) - 1:
            retval.append(newInd)
    return retval




# Day 1: Calorie Counting

## 1a. How many total Calories is the Elf who is carrying the most carrying?

## 1b. How many Calories are the three Elves who are carrying the most carrying in total?

In [22]:
# Parse input.  Sum values.  Sort values.

filename = 'prob01input.txt'
calories_list = get_string_input(filename)
elves = []    # list of elfPacks
elfPack = []  # list of integers
for x in calories_list:
    if len(x) == 0:  # indicates transition to new elf
        if len(elfPack) > 0:
            elves.append(elfPack)
            elfPack = []
        else:
            logging.error("Empty elfPack")
    else:
        elfPack.append(int(x))
elfTotals = [sum(x) for x in elves]
maxElfPack = max(elfTotals)
logging.info("1a.  Max Total Calories = {}".format(maxElfPack,))
topThreePack = sum(sorted(elfTotals, reverse=True)[:3])
logging.info("1b.  Sum Top Three Calories = {}".format(topThreePack,))

2023-01-24 09:38:18 INFO: Got 2244 lines from prob01input.txt.
2023-01-24 09:38:18 INFO: 1a.  Max Total Calories = 70613
2023-01-24 09:38:18 INFO: 1b.  Sum Top Three Calories = 205805


# Day 2: Rock Paper Scissors

## 2a.  What would your total score be in this case?

## 2b.  What would your total score be in this case?

In [23]:
# Conditional summation

filename = 'prob02input.txt'
rounds = get_string_input(filename)
rounds = [r.split(' ') for r in rounds]
#rounds = [['A','Y'],['B','X'],['C','Z']] #test input

def score_round_part_a(opp, me):
    retval = 'XYZ'.find(me) + 1  # points awarded for my choice
    # assum loss
    if [opp,me] in [['A','X'],['B','Y'],['C','Z']]:  #draw
        retval += 3
    elif [opp,me] in [['A','Y'],['B','Z'],['C','X']]: #win
        retval += 6
    return retval        

def score_round_part_b(opp, me):
    retval = 'XYZ'.find(me) * 3  # points award for lose, draw, win
    retval += 1  # assume rock
    if [opp,me] in [['A','Z'],['B','Y'],['C','X']]:  # points for paper
        retval += 1
    elif [opp,me] in [['A','X'],['B','Z'],['C','Y']]: # points for scisscors
        retval += 2
    return retval        


total_a = sum([score_round_part_a(r[0], r[1]) for r in rounds])
logging.info("2a. Total score = {}".format(total_a,))
total_b = sum([score_round_part_b(r[0], r[1]) for r in rounds])
logging.info("2b. Total score = {}".format(total_b,))


2023-01-24 09:38:19 INFO: Got 2500 lines from prob02input.txt.
2023-01-24 09:38:19 INFO: 2a. Total score = 10994
2023-01-24 09:38:19 INFO: 2b. Total score = 12526


# Day 3: Rucksack Reorganization

## 3a. What is the sum of the priorities of the mispacked item types?

## 3b. What is the sum of the priorities of bagde item types?

In [24]:
# Set intersection.

filename = 'prob03input.txt'
priorities = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

rucksacks = get_string_input(filename)
#rucksacks = ['vJrwpWtwJgWrhcsFMMfFFhFp','jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL',
#             'PmmdzqPrVvPwwTWBwg','wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn',
#             'ttgJtRGJQctTZtZT','CrZsJsPPZsGzwwsLwLmpwMDw']

values = []
for r in rucksacks:
    k = int(len(r)/2)
    p = set(r[:k]).intersection(set(r[k:])).pop()
    values.append(priorities.find(p) + 1)
logging.info("3a. Sum of mispacted priorities: {}".format(sum(values),))

values = []
groupCount = 0
for r in rucksacks:
    if groupCount == 0:
        items = set(r)
        groupCount = 1
    elif groupCount == 1:
        items = items.intersection(set(r))
        groupCount = 2
    else:
        p = items.intersection(set(r)).pop()
        items = set()
        groupCount = 0
        values.append(priorities.find(p) + 1)
logging.info("3b. Sum of badge priorities: {}".format(sum(values),))



2023-01-24 09:38:21 INFO: Got 300 lines from prob03input.txt.
2023-01-24 09:38:21 INFO: 3a. Sum of mispacted priorities: 7903
2023-01-24 09:38:21 INFO: 3b. Sum of badge priorities: 2548


# Day 4:  Camp Cleanup

## 4a. In how many assignment pairs does one range fully contain the other?

## 4b. In how many assignment pairs do the ranges overlap?

In [25]:
# Counting interval overlaps.

filename = 'prob04input.txt'
lines = get_string_input(filename)
#lines = ['2-4,6-8','2-3,4-5','5-7,7-9','2-8,3-7','6-6,4-6','2-6,4-8']

# Cleaning data
pairs = [l.split(',') for l in lines]                        # [['2-4','6-8'],...]
pairs = [[p[0].split('-'), p[1].split('-')] for p in pairs]  # [[['2','4'],['6','8']],...]
pairs = [[[int(e[0]), int(e[1])] for e in p] for p in pairs] # [[[2,4],[6,8]],...]

contained = 0
for e, f in pairs:
    if e[0] <= f[0] and f[1] <= e[1]:
        contained += 1
    elif f[0] <= e[0] and e[1] <= f[1]:
        contained += 1

overlap = sum([1 for e, f in pairs if not (e[1] < f[0] or f[1] < e[0])])
        
logging.info("4a: Completely contained = {}".format(contained,))
logging.info("4b: Some overlap = {}".format(overlap,))


2023-01-24 09:38:23 INFO: Got 1000 lines from prob04input.txt.
2023-01-24 09:38:23 INFO: 4a: Completely contained = 556
2023-01-24 09:38:23 INFO: 4b: Some overlap = 876


# Day 5:  Supply Stacks

## 5a. After rearrangement, what crate ends up on top of each stack?

## 5b. After rearrangement, what crate ends up on top of each stack?

In [26]:
# Follow a list of instructions.
# Stacks:  pop and append.  part II, list slicing

filename = 'prob05input.txt'
lines = get_string_input(filename)
#lines = ['    [D]    ','[N] [C]    ','[Z] [M] [P]',' 1   2   3 ',
#         '','move 1 from 2 to 1','move 3 from 1 to 3',
#         'move 2 from 2 to 1','move 1 from 1 to 2']

move_re = re.compile('^move (\d+) from (\d+) to (\d+)$')

# Parsing Input (probably should have hardcoded)
stacks = []
moves = []
stack_lines = True
top_stacks = True
for l in lines:
    if l == '':
        stack_lines = False
    elif stack_lines == True:
        # still creating stacks
        for i in range(1,len(l),4):
            if top_stacks == True:
                stacks.append([])
            if l[i] == '1':
                break # finished creating stacks
            elif l[i] == ' ':
                continue  # nothing on stack yet
            else:
                stacks[(i - 1)//4].append(l[i]) #add to stack
        top_stacks = False
    else:
        # finished creating stacks, parsing and storing moves
        m1 = move_re.match(l)
        if m1 == None:
            logging.error("Could not process line. {}".format(l,))
        else:
            moves.append([int(m1.groups()[0]),int(m1.groups()[1]),int(m1.groups()[2])])
    
stacks = [[x for x in reversed(s)] for s in stacks]
# stacks = [['Z','N'],['M','C','D'],'P']    # in test
# moves = [[1,2,1],[3,1,3],[2,2,1],[1,1,2]] # in test

orignal = [[x for x in s] for s in stacks] #deep copy

# Part A
for count, fromStack, toStack in moves:
    for i in range(count):
        val = stacks[fromStack-1].pop()
        stacks[toStack-1].append(val)
topsA = ''.join([s[-1] for s in stacks])
    
# Part B
stacks = orignal  # reset
for count, fromStack, toStack in moves:
    stacks[toStack-1] += stacks[fromStack-1][-1*count:]
    stacks[fromStack-1] = stacks[fromStack-1][:(-1)*count]
topsB = ''.join([s[-1] for s in stacks])

logging.info("5a. The crates on top: {}".format(topsA,))
logging.info("5b. The crates on top: {}".format(topsB,))

2023-01-24 09:38:25 INFO: Got 512 lines from prob05input.txt.
2023-01-24 09:38:25 INFO: 5a. The crates on top: FCVRLMVQP
2023-01-24 09:38:25 INFO: 5b. The crates on top: RWLWGJGFD


# Day 6:  Tuning Trouble

## 6a.  How many characters before the first start-of-packet marker is detected?

## 6b.  How many characters before the first start-of-message marker is detected?

In [27]:
# Reminds me of internet protocols
# Used list slicing and sets

filename = 'prob06input.txt'
lines = get_string_input(filename)
#lines =['mjqjpqmgbljsphdztnvjfqwrcgsmlb','bvwbjplbgvbhsrlpgdmjqwftvncz',
#        'nppdvjthqldpwncqszvftbrmjlhg','nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg',
#        'zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw']

for l in lines:
    partA = [i+4 for i in range(len(l)-4) if len(set(l[i:i+4])) == 4][0]
    partB = [i+14 for i in range(len(l)-14) if len(set(l[i:i+14])) == 14][0]
    logging.info("6a. The first start-of-packet ends at {}".format(partA,))
    logging.info("6b. The first start-of-message ends at {}".format(partB,))


2023-01-24 09:38:26 INFO: Got 1 lines from prob06input.txt.
2023-01-24 09:38:26 INFO: 6a. The first start-of-packet ends at 1655
2023-01-24 09:38:26 INFO: 6b. The first start-of-message ends at 2665


# Day 7:  No Space Left On Device

## 7a.  What is the sum of the directories with total size less than 100000?

## 7b.  What is the size of the smallest directory that would free up enough space?

In [28]:
# Tree structure, Recursion
# Knowledge of Linux helpful

filename = 'prob07input.txt'
lines = get_string_input(filename)
#lines = ['$ cd /','$ ls','dir a','14848514 b.txt','8504156 c.dat',
#         'dir d','$ cd a','$ ls','dir e','29116 f','2557 g','62596 h.lst',
#         '$ cd e','$ ls','584 i','$ cd ..','$ cd ..','$ cd d','$ ls',
#         '4060174 j','8033020 d.log','5626152 d.ext','7214296 k']

command_re = re.compile('^\$ (.*)$')
file_re = re.compile('^(\d+) (.*)$')
dir_re = re.compile('^dir (.*)$')

def get_dir_path(filesys, dirpath):
    ''' Given the filesys, navigate to and return the dictionary at dirpath'''
    retval = filesys
    for d in dirpath:
        retval = retval[d]
    return retval

filesys = {}  # Holds directory structure and contents.  
sizes = {}    # Holds directory sizes of each dirpath.
myDir = {}    # Pointer to current working directory.
myPath = []   # List of directories leading to current working directory.

# Parsing input (or is it output?)
for l in lines:
    m1 = command_re.match(l)
    m2 = file_re.match(l)
    m3 = dir_re.match(l)
    if m1 != None:  # Parse command line
        com = m1.groups(1)[0]
        if com == 'ls':
            pass
        elif com == 'cd /':  # navigate to top
            myPath, myDir = [], filesys
        elif com == 'cd ..':   # navigate up
            if len(myPath) > 0:
                myPath, myDir = myPath[:-1], get_dir_path(filesys, myPath)
        elif com[:3] == 'cd ':     # navigate down
            myPath.append(com[3:])
            myDir = get_dir_path(filesys, myPath) #set cwd
        else:
            logging.error("Unknown command".format(com,))
    elif m2 != None:  # file... save it and its size
        myDir[m2.groups(1)[1]] = int(m2.groups(1)[0])
    elif m3 != None:  # directory ... save it and create a dict for it
        myDir[m3.groups(1)[0]] = {}
    else:
        logging.error("Unable to parse input. '{}'".format(l,))
#logging.info("{}".format(filesys,))
# Finished parsing input (output?)

def get_dir_size(myPath, myDir):
    ''' Recursive function to get sum of all objects within directory.'''
    retval = 0
    for name, contents in myDir.items():
        if type(contents) == int:
            retval += contents
        elif type(contents) == dict:
            # Note the recursive call here
            retval += get_dir_size(myPath+[name], contents)
    sizes['/'.join(myPath)] = retval  # Save sizes to external object
    return retval

# Get sizes of all directories and store in sizes dict
_ = get_dir_size([''], filesys)

# 7a. Get sum of all directory sizes less than 100000
partA = sum([y for x, y in sizes.items() if y <= 100000])

# 7b. Get smallest directory that will free up enough space; 
# (7 - total + y >= 3)    iff    (y >= total - 4)
partB = min([y for x, y in sizes.items() if y >= sizes['']-40000000])

logging.info("7a. The sum of small directory sizes is {}".format(partA,))
logging.info("7b. The smallest directory size with enough space is {}".format(partB,))


2023-01-24 09:38:28 INFO: Got 1030 lines from prob07input.txt.
2023-01-24 09:38:28 INFO: 7a. The sum of small directory sizes is 1477771
2023-01-24 09:38:28 INFO: 7b. The smallest directory size with enough space is 3579501


# Day 8:  Treetop Tree House

## 8a.  How many trees are visible from outside the grid?

## 8b.  What is the maximum scenic score?

In [29]:
# Working in grids, topology rotation (not implemented by me)

filename = 'prob08input.txt'
lines = get_string_input(filename)
#lines = ['30373','25512','65332','33549','35390']
grid = [[int(x) for x in l] for l in lines]

def is_visible_from_left(row, col, grid):
    for c in range(col):
        if grid[row][c] >= grid[row][col]:
            return False
    return True

def is_visible_from_right(row, col, grid):
    for c in range(len(grid[0])-1, col, -1):
        if grid[row][c] >= grid[row][col]:
            return False
    return True

def is_visible_from_top(row, col, grid):
    for r in range(row):
        if grid[r][col] >= grid[row][col]:
            return False
    return True

def is_visible_from_bottom(row, col, grid):
    for r in range(len(grid)-1, row, -1):
        if grid[r][col] >= grid[row][col]:
            return False
    return True


def get_viewing_distance_left(row, col, grid):
    retval = 0
    for c in range(col-1, -1, -1):
        retval += 1
        if grid[row][c] >= grid[row][col]:  break
    return retval

def get_viewing_distance_right(row, col, grid):
    retval = 0
    for c in range(col+1, len(grid[0])):
        retval += 1
        if grid[row][c] >= grid[row][col]:  break
    return retval

def get_viewing_distance_top(row, col, grid):
    retval = 0
    for r in range(row-1, -1, -1):
        retval += 1
        if grid[r][col] >= grid[row][col]:  break
    return retval

def get_viewing_distance_bottom(row, col, grid):
    retval = 0
    for r in range(row+1, len(grid)):
        retval += 1
        if grid[r][col] >= grid[row][col]:  break
    return retval

# Part A
visible = 0
for r in range(len(grid)):
    for c in range(len(grid[r])):
        if is_visible_from_left(r, c, grid):     visible += 1
        elif is_visible_from_right(r, c, grid):  visible += 1
        elif is_visible_from_top(r, c, grid):    visible += 1
        elif is_visible_from_bottom(r, c, grid): visible += 1
            
# Part B
scenicScore = 0
for r in range(len(grid)):
    for c in range(len(grid[r])):
        lt = get_viewing_distance_left(r, c, grid)
        rt = get_viewing_distance_right(r, c, grid)
        tp = get_viewing_distance_top(r, c, grid)
        bm = get_viewing_distance_bottom(r, c, grid)
        scenicScore = max([lt * rt * tp * bm, scenicScore])
        
logging.info("8a. The number visible trees: {}".format(visible,))
logging.info("8b. The scenic score: {}".format(scenicScore,))

2023-01-24 09:38:31 INFO: Got 99 lines from prob08input.txt.
2023-01-24 09:38:31 INFO: 8a. The number visible trees: 1688
2023-01-24 09:38:31 INFO: 8b. The scenic score: 410400


# Day 9:  Rope Bridge

## 9a.  How many positions does the tail of a rope visit at least once?

## 9b.  How many positions does the tail of a larger rope visit at least once?

In [30]:
# Working in grids again.  Part II was fun.

filename = 'prob09input.txt'
lines = get_string_input(filename)
#lines = ['R 4','U 4','L 3','D 1','R 4','D 1','L 5','R 2']
#lines = ['R 5','U 8','L 8','D 3','R 17','D 10','L 25','U 20']
moves = [[l.split(' ')[0], int(l.split(' ')[1])] for l in lines] 

def move_head_knot(h, move):
    if move == 'R':  h[0] += 1
    elif move == 'L':  h[0] -= 1
    elif move == 'U':  h[1] += 1
    elif move == 'D':  h[1] -= 1
    return h   

def move_tail_knot(head, tail):
    hDist, vDist = head[0] - tail[0], head[1] - tail[1]
    if abs(hDist) > 1 or abs(vDist) > 1:
        tail = [tail[0] + np.sign(hDist), tail[1] + np.sign(vDist)]
    return tail

# Part A
h, t, aTails = [0,0], [0,0], set()
for move, count in moves:
    for _ in range(count):
        h = move_head_knot(h, move)
        t = move_tail_knot(h, t)
        aTails.add(tuple(t))

# Part B
knots, bTails = [[0,0] for i in range(10)], set()
for move, count in moves:
    for _ in range(count):
        knots[0] = move_head_knot(knots[0], move)
        for k in range(1, len(knots)):  # move subsequent knots
            knots[k] = move_tail_knot(knots[k-1], knots[k])
        bTails.add(tuple(knots[-1]))
        
logging.info("9a. Number of tail positions for 2 knots = {}".format(len(aTails),))
logging.info("9b. Number of tail positions for 10 knots = {}".format(len(bTails),))


2023-01-24 09:38:34 INFO: Got 2000 lines from prob09input.txt.
2023-01-24 09:38:34 INFO: 9a. Number of tail positions for 2 knots = 6464
2023-01-24 09:38:34 INFO: 9b. Number of tail positions for 10 knots = 2604


# Day 10:  Cathode-Ray Tube

## 10a.  What is the sum of six specific signal strengths?

## 10b.  What eight capital letters appear on your CRT?

In [31]:
# Follow the instructions.  Modular Arithmetic 

filename = 'prob10input.txt'
lines = get_string_input(filename)

'''lines = ['addx 15','addx -11','addx 6','addx -3','addx 5','addx -1','addx -8','addx 13','addx 4','noop',
 'addx -1','addx 5','addx -1','addx 5','addx -1','addx 5','addx -1','addx 5','addx -1','addx -35',
 'addx 1','addx 24','addx -19','addx 1','addx 16','addx -11','noop','noop','addx 21','addx -15',
 'noop','noop','addx -3','addx 9','addx 1','addx -3','addx 8','addx 1','addx 5','noop','noop',
 'noop','noop','noop','addx -36','noop','addx 1','addx 7','noop','noop','noop','addx 2','addx 6',
 'noop','noop','noop','noop', 'noop','addx 1','noop','noop','addx 7','addx 1','noop','addx -13',
 'addx 13','addx 7','noop','addx 1','addx -33','noop','noop','noop','addx 2','noop','noop','noop',
 'addx 8','noop','addx -1','addx 2','addx 1','noop','addx 17','addx -9','addx 1','addx 1','addx -3',
 'addx 11','noop','noop','addx 1','noop','addx 1','noop','noop','addx -13','addx -19','addx 1',
 'addx 3','addx 26','addx -30','addx 12','addx -1','addx 3','addx 1','noop','noop','noop','addx -9',
 'addx 18','addx 1','addx 2','noop','noop','addx 9','noop','noop','noop','addx -1','addx 2',
 'addx -37','addx 1','addx 3','noop','addx 15','addx -21','addx 22','addx -6','addx 1','noop',
 'addx 2','addx 1','noop','addx -10','noop','noop','addx 20','addx 1','addx 2','addx 2','addx -6',
 'addx -11','noop','noop','noop']
'''
#lines = ['noop','addx 3','addx -5']

commands = []
for l in lines:
    parts = l.split(' ')
    if len(parts) == 1:
        commands.append(parts)
    elif len(parts) == 2:
        commands.append([parts[0], int(parts[1])])
#commands = [['noop'],['addx',3],['addx',-5]]

# Get list of [interval, xReg]; store in steps
xReg, intervals, steps = 1, 1, []
steps.append([intervals, xReg])
for c in commands:
    intervals += 1
    if c[0] == 'noop':
        steps.append([intervals, xReg])
    elif c[0] == 'addx':
        steps.append([intervals, xReg])
        intervals += 1
        xReg += c[1]
        steps.append([intervals, xReg])
# steps = [[1, val at 1], [2, val at 2], ...]

partA = 0
for s in [20, 60, 100, 140, 180, 220]:
    partA += s * steps[s-1][1]
    
partB = ''
for s in steps:
    if ((s[0]%40) - s[1]) in [0, 1, 2]:
        partB += '#'
    else:
        partB += '.'
    
        
logging.info("10a. Total syntax error score = {}".format(partA,))
logging.info("10b. = ")
for x in [0, 40, 80, 120, 160, 200]:
    logging.info("{}".format(partB[x:x+40]),)


2023-01-24 09:38:36 INFO: Got 145 lines from prob10input.txt.
2023-01-24 09:38:36 INFO: 10a. Total syntax error score = 12560
2023-01-24 09:38:36 INFO: 10b. = 
2023-01-24 09:38:36 INFO: ###..#....###...##..####.###...##..#...#
2023-01-24 09:38:36 INFO: #..#.#....#..#.#..#.#....#..#.#..#.#....
2023-01-24 09:38:36 INFO: #..#.#....#..#.#..#.###..###..#....#....
2023-01-24 09:38:36 INFO: ###..#....###..####.#....#..#.#....#....
2023-01-24 09:38:36 INFO: #....#....#....#..#.#....#..#.#..#.#....
2023-01-24 09:38:36 INFO: #....####.#....#..#.#....###...##..####.


# Day 11:  Monkey in the Middle

## 11a.  What is the level of monkey business after 20 rounds?

## 11b.  What is the level of monkey business after 10000 rounds?

In [32]:
# Class structures for Monkeys helpful.
# Knowledge of Modular Arithmetic helpful.
# Hints of the Chinese Remainder Theorem.

class Monkey:
    def __init__(self, worryModulus, trueTarget, falseTarget, opFunc, start_items):
        self.worryModulus = worryModulus
        self.trueTarget = trueTarget   # pass to monkey trueTarget if worry_test == True
        self.falseTarget = falseTarget # pass to monkey falseTarget if worry_test == False
        self.operation = opFunc
        self.items = start_items
        self.numInspected = 0
        self.adjust_worry = self.adjust_worry_A
        self.groupWorryModulus = None  # For part B
        
    def worry_test(self, value):
        return (value % self.worryModulus == 0)

    def adjust_worry_A(self, x):
        return math.floor(self.operation(x)/3)
    
    def adjust_worry_B(self, x):
        # Statement: If x % m = r and m divides g then (x % g) % m = r.
        # Proof:
        # Let x % g = s.  Need to show s % m = r.  
        # By definition of modulo there exists integer z and y such that
        #    x = mz + r  (1)    and     x = gy + s  (2).
        # Substituting (2) into (1) and solving for s gives
        #    s  = mz - gy + r   (3).
        # and since m divides g then there exists an integer l such that
        #    g = ml    (4).
        # Substituting (4) into (3) gives 
        #    s = mz - mly + r   (5)
        # and factoring out an m gives
        #    s = m(z - ly) + r   (6)
        # Since (z - ly) is an integer, by definition s % m = r.  Q.E.D.
        
        # This means that that x can continually be reduced by
        #  the groupWorryModulus (to keep it from getting too large)
        #  and the worry_test for each monkey will still hold.
        return self.operation(x) % self.groupWorryModulus
    
    def examine_items(self):
        targets = []
        for x in self.items:
            worry = self.adjust_worry(x)
            t = self.falseTarget
            if self.worry_test(worry):  
                t = self.trueTarget
            targets.append([t, worry])
            self.numInspected += 1
        self.items = []
        return targets


# Hard coded inputs
def get_init_monkeys(test):
    monkeys = []
    if test == True:
        monkeys.append(Monkey(23, 2, 3, lambda x: x*19, [79,98]))
        monkeys.append(Monkey(19, 2, 0, lambda x: x+ 6, [54,65,75,74]))
        monkeys.append(Monkey(13, 1, 3, lambda x: x* x, [79,60,97]))
        monkeys.append(Monkey(17, 0, 1, lambda x: x+ 3, [74]))
    else:
        monkeys.append(Monkey( 7, 6, 2, lambda x: x*19, [59,74,65,86]))
        monkeys.append(Monkey( 2, 2, 0, lambda x: x+ 1, [62,84,72,91,68,78,51]))
        monkeys.append(Monkey(19, 6, 5, lambda x: x+ 8, [78,84,96]))
        monkeys.append(Monkey( 3, 1, 0, lambda x: x* x, [97,86]))
        monkeys.append(Monkey(13, 3, 1, lambda x: x+ 6, [50]))
        monkeys.append(Monkey(11, 4, 7, lambda x: x*17, [73,65,69,65,51]))
        monkeys.append(Monkey( 5, 5, 7, lambda x: x+ 5, [69,82,97,93,82,84,58,63]))
        monkeys.append(Monkey(17, 3, 4, lambda x: x+ 3, [81,78,82,76,79,80]))
    return monkeys


# Part A
roundsA = 20
test = False
monkeys = get_init_monkeys(test)

# Run rounds
for r in range(roundsA):
    for m in range(len(monkeys)):
        # for each monkey, examine all items, adjust worry, and distribute
        for newM, val in monkeys[m].examine_items():
            monkeys[newM].items.append(val)
countsA = sorted([monkey.numInspected for monkey in monkeys], reverse=True)

# Part B
roundsB = 10000
test = False
monkeys = get_init_monkeys(test)

# Adjust monkey parameters for part B (See proof in Monkey.adjust_worry_B)
groupWorryModulus = np.prod([m.worryModulus for m in monkeys])
for m in monkeys:
    m.groupWorryModulus = groupWorryModulus
    m.adjust_worry = m.adjust_worry_B

# Run rounds
for r in range(roundsB):
    for m in range(len(monkeys)):
        # for each monkey, examine all items, adjust worry, and distribute
        for newM, val in monkeys[m].examine_items():
            monkeys[newM].items.append(val)
countsB = sorted([monkey.numInspected for monkey in monkeys], reverse=True)

logging.info("11a. Monkey business after {} rounds: {}.".format(roundsA, countsA[0]*countsA[1],))
logging.info("11b. Monkey business after {} rounds: {}.".format(roundsB, countsB[0]*countsB[1],))


2023-01-24 09:38:39 INFO: 11a. Monkey business after 20 rounds: 61005.
2023-01-24 09:38:39 INFO: 11b. Monkey business after 10000 rounds: 20567144694.


# Day 12:  Hill Climbing Algorithm

## 12a.  What is the length of the shortest from S to E?

## 12b.  What is the length of the shortest from any a to E?

In [33]:
# Dijkstra's Algorithm
# Uses get_grid_neighborhood and get_grid_index from initial
#  set of functions to implement grid topology.

filename = 'prob12input.txt'
lines = get_string_input(filename)
#lines = ['Sabqponm','abcryxxl','accszExk','acctuvwj','abdefghi']
heights = 'SabcdefghijklmnopqrstuvwxyzE'

def is_legal_step(heights, cur, neighbor):
    return heights.find(neighbor) - heights.find(cur) <= 1


def find_shortest_path(A, n, m, start, end):
    ''' Implement Dijkstra's Algorithm on an unweighted graph.
        Graph vertices are grid squares with edges defined by
        is_legal_step().  Uses get_grid_neighborhood().
    '''
    retval = None
    visited = set()  # The vertices that have been visited
    distances = {start:0}  # dict holding current shortest path lengths to vertices
    possibles = [start]  # List of unvisited vertices connected to visited vertices
    while len(possibles) > 0:
        cur = possibles.pop(0)        # consider first possible state, call it cur
        dist_to_cur = distances[cur]  # get current distance to cur... this is shortest possible
        visited.add(cur)              # visit cur
        if cur == end:                # early break condition  
            retval = dist_to_cur
            break

        # else: add each unvisited neighbor of cur to possibles
        for ind in get_grid_neighborhood(cur, n, m, diagonal=False):
            if ind not in visited and is_legal_step(heights, A[cur], A[ind]):
                if distances.get(ind, None) == None:
                    distances[ind] = dist_to_cur + 1
                    possibles.append(ind)
                # This does not generalize: Should check if distances[ind] exist
                # and if unvisited, if so update/minimize distances[ind] if needed
                # Logic works here because of grid structure and unweighted stepping.

        # Sort list of possible vertices so vertex at shortest distance
        #  will be visited during next iteration.
        possibles = sorted(possibles, key=lambda x: distances[x])

    return retval


# Parsing input
A1, n1, m1 = dict(), len(lines), len(lines[0])
start, end = None, None
for r, c in tt.product(range(n1), range(m1)):
    index = get_grid_index(r, c, n1, m1)
    A1[index] = lines[r][c]
    if A1[index] == 'S':   start = index
    if A1[index] == 'E':   end = index

# Part a.
partA = find_shortest_path(A1, n1, m1, start, end)

# Part b
partB = partA  # partA can serve as an initial min
for index, val in A1.items():
    if val == 'a':
        lengthFromIndex = find_shortest_path(A1, n1, m1, index, end)
        if lengthFromIndex != None:
            partB = min([partB, lengthFromIndex])

logging.info('12a. Length of Shortest Path From S: {}'.format(partA,))
logging.info('12b. Length of Shortest Path From a: {}'.format(partB,))

2023-01-24 09:38:41 INFO: Got 41 lines from prob12input.txt.
2023-01-24 09:38:42 INFO: 12a. Length of Shortest Path From S: 352
2023-01-24 09:38:42 INFO: 12b. Length of Shortest Path From a: 345


# Day 13:  Distress Signal

## 13a.  What is the sum of the indices of the ordered pairs?

## 13b.  What is the decoder key for the distress signal?

In [34]:
# JSON, Recursion, Sorting with unconventional functions.

filename = 'prob13input.txt'
lines = get_string_input(filename)
#lines = ['[1,1,3,1,1]','[1,1,5,1,1]','','[[1],[2,3,4]]','[[1],4]',
#         '','[9]','[[8,7,6]]','','[[4,4],4,4]','[[4,4],4,4,4]','',
#         '[7,7,7,7]','[7,7,7]','','[]','[3]','','[[[]]]','[[]]','',
#         '[1,[2,[3,[4,[5,6,7]]]],8,9]','[1,[2,[3,[4,[5,6,0]]]],8,9]']

def compare_packets(p, q):
    ''' Recursive function comparing p and q.  
        Return retval in [-1,0,1] if [p<q, p==q, p>q], respectively.
    '''
    retval = 0
    if type(p) == int and type(q) == int:
        if p < q:
            retval = -1
        elif p > q:
            retval = 1

    elif type(p) == int and type(q) == list:
        retval = compare_packets([p], q)
        
    elif type(p) == list and type(q) == int:
        retval = compare_packets(p, [q])

    elif type(p) == list and type(q) == list:
        for i in range( min([len(p), len(q)])):
            retval = compare_packets(p[i], q[i])
            if retval != 0:
                break
        if retval == 0 and len(p) != len(q):
            if len(p) < len(q):
                retval = -1
            else:
                retval = 1

    return retval
 

# Load input
pairs = []
for i in range(0,len(lines),3):
    pairs.append([json.loads(lines[i]), json.loads(lines[i+1])])
    
# Part A
partA = 0
for i, [p, q] in enumerate(pairs):
    if compare_packets(p, q) == -1:
        partA += (i+1)

# Part B
partB = 0 # decoder key is product of divider packets
# Create list of allPairs
packets = [ [[2]], [[6]] ]  # the two divider packets
for p, q in pairs:
    packets.append(p)
    packets.append(q)

# Sort all packets
packets = sorted(packets, key=functools.cmp_to_key(compare_packets))

# Find and use indices of [[2]] and [[6]]
for i, p in enumerate(packets):
    if compare_packets([[2]], p) == 0:
        partB = i+1
    elif compare_packets([[6]], p) == 0:
        partB *= (i+1)

logging.info("13a. The sum of the indices of the ordered pairs {}".format(partA,))
logging.info("13b. The decoder key is: {}".format(partB,))
    

2023-01-24 09:38:46 INFO: Got 449 lines from prob13input.txt.
2023-01-24 09:38:46 INFO: 13a. The sum of the indices of the ordered pairs 5013
2023-01-24 09:38:46 INFO: 13b. The decoder key is: 25038


# Day 14:  Regolith Reservoir

## 14a.  How many units of sand come to rest before sand flows into the abyss below?

## 14b.  How many units of sand come to rest?

In [35]:
# Working on a grid.  Deciding how to build and use structures.
# This could be faster by using a queue for the path the sand takes.

filename = 'prob14input.txt'
lines = get_string_input(filename)
#lines = ['498,4 -> 498,6 -> 496,6','503,4 -> 502,4 -> 502,9 -> 494,9']

walls = []
for l in lines:
    points = [[int(i) for i in p.split(',')] for p in l.split(' -> ')]
    walls.append(points)
# walls = [ [[498,4],[498,6],[496,6]], [[503,4],[502,4],[502,9],[494,9]], ...]

# Create rocks set, set of all points on any line.
rocks = set()
for wall in walls:
    #logging.info("Creating wall {}".format(wall,))
    p = wall[0]
    for q in wall[1:]:
        if p[0] == q[0]:
            for y in range(min(p[1],q[1]),max(p[1],q[1])+1):
                rocks.add(tuple([p[0],y]))
        elif p[1] == q[1]:
            for x in range(min(p[0],q[0]),max(p[0],q[0])+1):
                rocks.add(tuple([x, p[1]]))
        p = q # set up for next iteration
# rocks = {(498,4),(498,5),(498,6),(497,6),(496,6),(503,4),...}

maxDepth = max([p[1] for p in rocks])
logging.info("max rock depth = {}".format(maxDepth,))

def move_down(curPos, rocks, blocked):
    ''' Drop sand one row if possible.  Return new position of sand
         if movement not blocked and return None if movement blocked.
    '''
    retval = None
    x,y = curPos
    for p in [(x,y+1), (x-1,y+1), (x+1,y+1)]:
        if p not in rocks and p not in blocked:
            retval = p
            break
    return retval


# Part A
blockedA = set()
sand = (500,0) # Enter loop first time
while sand[1] < maxDepth:
    sand = (500,0) # Reset fall
    while sand[1] < maxDepth:
        newSand = move_down(sand, rocks, blockedA)
        if newSand != None:
            sand = newSand
        else:
            blockedA.add(sand)
            break

# Part B
blockedB = set()
start = (500,0) 
while start not in blockedB: 
    sand = start # Reset fall
    while sand[1] < maxDepth + 2:
        newSand = move_down(sand, rocks, blockedB)
        if newSand == None:
            blockedB.add(sand)
            break
        elif newSand[1] >= maxDepth + 2:
            blockedB.add(sand)
            break
        else:
            sand = newSand

    
logging.info("14a. Number of grains of sand {}".format(len(blockedA), ))
logging.info("14b. Number of grains of sand {}".format(len(blockedB), ))


2023-01-24 09:38:50 INFO: Got 179 lines from prob14input.txt.
2023-01-24 09:38:50 INFO: max rock depth = 171
2023-01-24 09:38:52 INFO: 14a. Number of grains of sand 1068
2023-01-24 09:38:52 INFO: 14b. Number of grains of sand 27936


# Day 15:  Beacon Exclusion Zone

## 15a.  In the row where y=2000000, how many positions cannot contain a beacon?

## 15b.  What is its tuning frequency?

In [36]:
# Manhattan Distance.  Do some algebra to get formula.
# Efficient searching of large search space.

filename = 'prob15input.txt'
lines = get_string_input(filename)
targetLine = 2000000

'''lines = ['Sensor at x=2, y=18: closest beacon is at x=-2, y=15',
         'Sensor at x=9, y=16: closest beacon is at x=10, y=16',
         'Sensor at x=13, y=2: closest beacon is at x=15, y=3',
         'Sensor at x=12, y=14: closest beacon is at x=10, y=16',
         'Sensor at x=10, y=20: closest beacon is at x=10, y=16',
         'Sensor at x=14, y=17: closest beacon is at x=10, y=16',
         'Sensor at x=8, y=7: closest beacon is at x=2, y=10',
         'Sensor at x=2, y=0: closest beacon is at x=2, y=10',
         'Sensor at x=0, y=11: closest beacon is at x=2, y=10',
         'Sensor at x=20, y=14: closest beacon is at x=25, y=17',
         'Sensor at x=17, y=20: closest beacon is at x=21, y=22',
         'Sensor at x=16, y=7: closest beacon is at x=15, y=3',
         'Sensor at x=14, y=3: closest beacon is at x=15, y=3',
         'Sensor at x=20, y=1: closest beacon is at x=15, y=3']
targetLine = 10
'''

# Parsing input
line_re = re.compile('^Sensor at x=([-\d]+), y=([-\d]+): closest beacon is at x=([-\d]+), y=([-\d]+)$')
data = []
for l in lines:
    m1 = line_re.match(l)
    if m1 == None:
        logging.error("Could not parse line '{}'".format(l,))
    else:
        sx, sy = int(m1.groups(1)[0]), int(m1.groups(1)[1])
        bx, by = int(m1.groups(1)[2]), int(m1.groups(1)[3])
        data.append([[sx,sy],[bx,by]])


def get_beacon_free_spaces_range_on_line(d, targetLine):
    ''' Return range of xVals, [xMin, xMax], on y=targetLine that can
        be determined to be beacon-free based on datapoint d where
        d = [ [sensor x, sensor y], [closest beacon x, closest beacon y] ]
        and y = integer
    '''
    retval = []
    sx, sy = d[0]
    bx, by = d[1]
    mDist = abs(sx - bx) + abs(sy - by)
    yLineDist = abs(sy - targetLine)
    if yLineDist <= mDist:
        retval = [sx - (mDist-yLineDist), sx + (mDist-yLineDist)]
    return retval


beacons = set([tuple(d[1]) for d in data])

# Part A
freeSpaces = set()  # length will be solution.
xRanges = []
for d in data:
    xRange = get_beacon_free_spaces_range_on_line(d, targetLine)
    if len(xRange) == 2:   # Range exists, add values to free space     
        for x in range(xRange[0],xRange[1]+1):
            if (x,targetLine) not in beacons:  # don't add beacons
                freeSpaces.add(x)

                
# Part B
# There is most likely an faster way by examining the boundaries
#  of each sensor region and testing whether they are within another
#  sensor region.  As it is, my approach of examining each line and
#  getting the ranges of beacon free spaces on that line is fast 
#  enough for my liking.

#for target in range(0,21):
freq = -1
# Test each line as target until solution found
for target in range(0,4000001):
    if target % 500000 == 0:
        logging.info("part B target = {}".format(target,))
    # Get all ranges of free spaces on this target
    xRanges = []
    for d in data:
        #beacons.add(tuple(d[1]))
        xRange = get_beacon_free_spaces_range_on_line(d, target)
        if len(xRange) == 2:
            xRanges.append(xRange)

    # Examing free space ranges looking for a gap.
    xCur = -1
    for x1, x2 in sorted(xRanges):
        if x1 > xCur+1:
            logging.info("Open spaces from ({},{}) to ({},{})".format(xCur+1, target, x1-1, target))
            freq = (4000000 * (xCur+1)) + target
            break
        xCur = max([xCur, x2])
    if freq != -1:
        break

logging.info("15a. Positions that cannot contain a beacon: {}".format(len(freeSpaces),))
logging.info("15b. Tuning frequency: {}".format(freq,))


2023-01-24 09:38:54 INFO: Got 25 lines from prob15input.txt.
2023-01-24 09:38:56 INFO: part B target = 0
2023-01-24 09:39:03 INFO: part B target = 500000
2023-01-24 09:39:10 INFO: part B target = 1000000
2023-01-24 09:39:17 INFO: part B target = 1500000
2023-01-24 09:39:25 INFO: part B target = 2000000
2023-01-24 09:39:32 INFO: part B target = 2500000
2023-01-24 09:39:41 INFO: part B target = 3000000
2023-01-24 09:39:45 INFO: Open spaces from (3156345,3204261) to (3156345,3204261)
2023-01-24 09:39:45 INFO: 15a. Positions that cannot contain a beacon: 5688618
2023-01-24 09:39:45 INFO: 15b. Tuning frequency: 12625383204261


# Day 16:  Proboscidea Volcanium

## 16a.  What is the most pressure you can release?

## 16b.  What is the most pressure you can release with the elephant's help?

In [37]:
# Efficient searching.  Pruning.  DFS vs BFS.
# This was probably the most difficult day for me.  I did not initially
# realize that the graph itself could be condensed to a distance graph
# which was much more efficient to search at first.  My initial solution
# (which worked but was very messy) took 20 minutes to run.  This approach
# is much cleaner and takes a few seconds.

# Approach: 
# 1) Condense the graph to a distance graph using Dijkstra's algorithm
# to find the shortest distance between every pair of vertices.  Eliminate
# all vertices from the distance graph where the flow rate of valve is 0.
# This updated graph is stored in 'M' in the code below.
#
# 2) Use a recursive depth first search (DFS) to find all relevant paths
# through the distance graph.  Save all paths and the pressure they release,
# even intermediate paths as standing still is an option (note, this is
# important for part B).
# 
# 3) Sort paths by the pressure they release (descending).
#
# 4) For part A, the solution is the first path.
#
# 5) For part B, compare all paths to find the best non-overlapping pair.

filename = 'prob16input.txt'
lines = get_string_input(filename)

'''lines = ['Valve AA has flow rate=0; tunnels lead to valves DD, II, BB',
         'Valve BB has flow rate=13; tunnels lead to valves CC, AA',
         'Valve CC has flow rate=2; tunnels lead to valves DD, BB',
         'Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE',
         'Valve EE has flow rate=3; tunnels lead to valves FF, DD',
         'Valve FF has flow rate=0; tunnels lead to valves EE, GG',
         'Valve GG has flow rate=0; tunnels lead to valves FF, HH',
         'Valve HH has flow rate=22; tunnel leads to valve GG',
         'Valve II has flow rate=0; tunnels lead to valves AA, JJ',
         'Valve JJ has flow rate=21; tunnel leads to valve II']
'''
''' I-J
    |
    A-D-E-F-G-H
    | |
    B-C
'''

class Vertex:
    ''' Structure to hold the initial graph as well as the flow rates.'''
    def __init__(self, name, rate, neighbors):
        self.name = name
        self.rate = rate
        self.N = sorted(neighbors)


# Parsing input
line_re = re.compile("^Valve ([A-Z]{2}) .* rate=([\d]+); .* valves? ([A-Z, ]+)")
vertices = dict()
valvesActive = dict()
for l in lines:
    m1 = line_re.match(l)
    if m1 != None:
        v = Vertex(m1.groups(1)[0], int(m1.groups(1)[1]), m1.groups(1)[2].split(', ') )
        vertices[v.name] = v
        if v.rate > 0:
            valvesActive[v.name] = False
    else:
        logging.info("could not parse {}".format(l,))


def find_all_shortest_paths_from_vertex(source, V):
    ''' Use simplified Dijkstra's Algorithm to find length of 
        shortest paths from source to all other vertices in the 
        graph.  Eliminate any vertex from distances that has a
        flow rate of 0.
        Return a dictionary of distances; 
          retval[vertex] = shortest distance from source to vertex
        Note: this algorithm takes shortcuts since V is unweighted.
    '''
    distances = {source:0} # dict holding current shortest path lengths to vertices
    possibles = [source]   # List of unvisited vertices connected to visited vertices
    while len(possibles) > 0:
        cur = possibles.pop(0)        # consider first possible state, call it cur
        dist_to_cur = distances[cur]  # get current distance to cur... this is shortest possible

        # add each new neighbor of cur to possibles
        for neighbor in V[cur].N:  # for each neighbor
            if distances.get(neighbor, None) == None:
                # First time encountering neighbor, add to distances and possibles
                distances[neighbor] = dist_to_cur + 1
                possibles.append(neighbor)
                
    # Remove distances to vertices that have zero rate.
    retval = dict()
    for v in distances.keys():
        if V[v].rate > 0:
            retval[v] = distances[v]
    return retval


def find_all_shortest_paths(V):
    ''' Use Dijkstra's Algorithm to find the shortest path between
        each pair of vertices having valves with positive release.
    '''
    retval = dict()
    for v in V.keys():
        if V[v].rate > 0:
            retval[v] = find_all_shortest_paths_from_vertex(v, V)
    return retval
    
# Creating distance graph M; M[v1][v2] = shortest distance from v1 to v2
M = find_all_shortest_paths(vertices) 
# Add starting vertex to M even though flow rate at 'AA' is zero.
# Note that 'AA' does not appear as a key in any M[v].keys() 
M['AA'] = find_all_shortest_paths_from_vertex('AA', vertices) 

# Output to view structure M.
#for v1 in sorted(M.keys()):
#    dists = [[v2, M[v1][v2]] for v2 in sorted(M[v1].keys())]
#    logging.info("v1 = {}:  {}".format(v1, dists))


class State:
    ''' This is a helper state for the recursive function follow_path().
        It holds many of the variables that would be passed up and down
         the recursive tree.
    '''
    def __init__(self, time, active, loc, path=[]):
        self.time = time       # time = integer: time to traverse self.path
        self.active = active   # active[v] = Boolean
        self.loc = loc         # loc = string: current location
        self.path = path       # path = [v1, v2, ...] = path so far
        self.released = 0      # released = integer: pressure released so far
        self.allPaths = dict() # allPaths[path] = integer: pressure released by path
        

def follow_path(cur, maxTime, M, V):
    ''' DFS to examine all possible paths through M.
        Path saved to cur.allPaths along with the pressure released.
        cur -> holds all dynamic variables (State() object)
        maxTime = constant total time
        M = constant (shortest distances between vertices)
        V = original set of vertices; needed for flow rates        
    '''
    if cur.time < maxTime:
        # Time available to visit cur.loc and open valve
        curLoc = cur.loc  # save for later
        scoreAtCur = V[curLoc].rate * (maxTime - cur.time)
        cur.released += scoreAtCur 
        cur.active[curLoc] = True
        cur.path.append(curLoc)
        cur.allPaths[tuple(cur.path)] = cur.released  # save cur.path

        # Consider each unvisited vertex (possible child)
        for v in M[curLoc].keys():
            if cur.active.get(v, True) == False:
                cur.time += M[curLoc][v] + 1    # add time to open v
                cur.loc = v                     # go try to visit v
                follow_path(cur, maxTime, M, V) # recursive call
                cur.time -= (M[curLoc][v] + 1)  # remove time to open v
            # Finished visiting v
        # Finished visiting children

        
        # Reset dynamic values of cur before returning
        cur.active[curLoc] = False
        cur.loc = curLoc 
        cur.released -= scoreAtCur 
        _ = cur.path.pop(-1)

# Part A
maxTime = 30
active = dict()
for v in M['AA'].keys():
    active[v] = False
cur = State(0, active, 'AA', path=[])
follow_path(cur, maxTime, M, vertices)
logging.info("Number of Paths found for A: {}".format(len(cur.allPaths)))

sorted_paths = sorted(cur.allPaths.keys(), 
                      key=lambda x: cur.allPaths[x], 
                      reverse=True)
partA, bestPathForA = cur.allPaths[sorted_paths[0]], sorted_paths[0]
logging.info("16a. {} released by path {}".format(partA, bestPathForA))


# Part B
maxTime = 26
active = dict()
for v in M['AA'].keys():
    active[v] = False
cur = State(0, active, 'AA', path=[])
follow_path(cur, maxTime, M, vertices)
logging.info("Number of Paths found for B: {}".format(len(cur.allPaths)))

partB, bestPathForB = 0, [None, None]
# sorting helps narrow search for optimal pair
sorted_paths = sorted(cur.allPaths.keys(), 
                      key=lambda x: cur.allPaths[x], 
                      reverse=True)

# Find pair of paths with highest released sum that only intersect at 'AA'
for i, path1 in enumerate(sorted_paths):
    visited1 = set(path1[1:])  # [1:] removes 'AA'
    
    for path2 in sorted_paths[i+1:]:
        # check if sum of released flows could be best possible
        if cur.allPaths[path1] + cur.allPaths[path2] <= partB:
            break  # note that paths are sorted

        # check for overlap in path1 and path2 
        elif len(set(path2[1:]).intersection(visited1)) == 0:
            partB = cur.allPaths[path1] + cur.allPaths[path2]
            bestPathForB = [path1, path2]
            
logging.info("16b. {} released by paths {}".format(partB, bestPathForB))
# Note that there is time for the second path to be longer by going 
# from YV to EB and opening that valve.  However, EB is on the first
# path.  This is why all intermediate paths must be saved and not
# just paths of maximal length.


2023-01-24 09:39:50 INFO: Got 61 lines from prob16input.txt.
2023-01-24 09:39:51 INFO: Number of Paths found for A: 180877
2023-01-24 09:39:52 INFO: 16a. 1947 released by path ('AA', 'IY', 'XF', 'IU', 'JF', 'JG', 'QH', 'SZ', 'BF')
2023-01-24 09:39:52 INFO: Number of Paths found for B: 42631
2023-01-24 09:39:55 INFO: 16b. 2556 released by paths [('AA', 'EB', 'JF', 'JG', 'QH', 'SZ', 'BF', 'ZB'), ('AA', 'IY', 'XF', 'IU', 'CY', 'YV')]


In [ ]:
# This was my initial attempt at a solution, which did actually work but
# is a mess and takes ~15 minutes to finish.  The approach uses a
# BFS method and prunes as it goes along.  I left it here in case anyone
# was interested, but the previous cell is a much better solution.

# This code is a mess... an indication that I struggled to hack together a solution.

filename = 'prob16input.txt'
lines = get_string_input(filename)

'''lines = ['Valve AA has flow rate=0; tunnels lead to valves DD, II, BB',
         'Valve BB has flow rate=13; tunnels lead to valves CC, AA',
         'Valve CC has flow rate=2; tunnels lead to valves DD, BB',
         'Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE',
         'Valve EE has flow rate=3; tunnels lead to valves FF, DD',
         'Valve FF has flow rate=0; tunnels lead to valves EE, GG',
         'Valve GG has flow rate=0; tunnels lead to valves FF, HH',
         'Valve HH has flow rate=22; tunnel leads to valve GG',
         'Valve II has flow rate=0; tunnels lead to valves AA, JJ',
         'Valve JJ has flow rate=21; tunnel leads to valve II']
'''
''' I-J
    |
    A-D-E-F-G-H
    | |
    B-C
'''

class Vertex:
    def __init__(self, name, rate, neighbors):
        self.name = name
        self.rate = rate
        self.N = sorted(neighbors)
        
    def display(self):
        logging.info("Valve {}, rate {}, N = {}".format(self.name, self.rate, self.N))


# Parsing input
line_re = re.compile("^Valve ([A-Z]{2}) .* rate=([\d]+); .* valves? ([A-Z, ]+)")
vertices = dict()
valvesActive = dict()
for l in lines:
    m1 = line_re.match(l)
    if m1 != None:
        v = Vertex(m1.groups(1)[0], int(m1.groups(1)[1]), m1.groups(1)[2].split(', ') )
        vertices[v.name] = v
        if v.rate > 0:
            valvesActive[v.name] = False
    else:
        logging.info("could not parse {}".format(l,))


def all_valves_open(A):
    ''' Boolean to check if all helpful valves are open. '''
    return all([ A[v] for v in A.keys() ])


class State:
    def __init__(self, t, A, loc, rate=0, rel=0, mn=0, mx=0):
        self.time = t        # self.time = integer
        self.active = A      # self.active[v] = Boolean
        self.loc = loc       # self.loc = location (could be string or list of strings)
        self.rate = rate     # self.rate = integer; current rate of release
        self.released = rel  # self.released = integer; current pressure released
        self.minPoten = mn   # self.minPoten = integer; minimum potential released
        self.maxPoten = mx   # self.maxPoten = integer; maximum potential released

    def _copy(self):
        return State(self.t, self.active.copy, self.rate, self.released, self.minPoten, self.maxPoten)

    def pass_time(self):
        self.time += 1
        self.released += rate
        # Update potential?
        
    def open_valve(self, v, V):
        if self.active.get(v, True) == False:
            self.active[v] = True
            self.rate += V[v].rate  # V is the set of Vertex objects
        # Update potential?
        
    
def make_next_moves(V, possibles, numRemain, maxRoundReleased):
    ''' V = vertices in cave;  V[name] = Vertex object
        possibles = (time, vertex, released, rate, activeValves, [])
        numRemain is the number of remaining rounds for each state
        maxRoundReleased is the number of pressure released if all valves are open.
    '''
    retval = dict()
    allValvesOpen = -1
    maxMinPotential = -1
    
    for state in possibles:
        newState, newVals = None, None
        time, v, released, rate, A, P = state
        newReleased = released + rate
        minPotential = newReleased + (numRemain*rate)
        maxPotential = newReleased + (numRemain*maxRoundReleased)
        maxMinPotential = max([minPotential, maxMinPotential])
        if maxPotential < maxMinPotential:
            # The max potential of this state is below the minimal potential
            #  of some other state, so abandon this path.
            continue
        newTime = time + 1

        if newReleased < allValvesOpen:
            continue
        
        if all_valves_open(A):
            # this state has all valves open
            newState = tuple([newTime, v] + [A[x] for x in sorted(A.keys())])
            newVals = [newTime, v, newReleased, rate, A.copy(), P.copy() + ['Pass']]
            prevVals = retval.get(newState, None)
            if prevVals == None or newReleased > prevVals[2]:
                retval[newState] = newVals
                allValvesOpen = max([allValvesOpen, newReleased])
            continue
        
        # Make next moves, open or move to neighbor
        for action in ['open'] + V[v].N:
            newState, newVals = None, None
                
            if action == 'open' and A.get(v, True) == False:
                Acopy = A.copy()
                Acopy[v] = True
                newRate = rate + V[v].rate
                newState = tuple([newTime, v] + [Acopy[x] for x in sorted(Acopy.keys())])
                newVals = [newTime, v, newReleased, newRate, Acopy.copy(), P.copy() + ['Open']]
                    
            elif action != 'open':
                newState = tuple([newTime, action] + [A[x] for x in sorted(A.keys())])
                newVals = [newTime, action, newReleased, rate, A.copy(), P.copy() + [action]] 

            if newState != None:
                prevVals = retval.get(newState, None)
                if prevVals == None:             retval[newState] = newVals
                elif newReleased > prevVals[2]:  retval[newState] = newVals

    return retval.values()


def make_next_moves2(V, possibles, numRemain, maxRoundReleased):
    ''' V = vertices in cave;  V[name] = Vertex object
        possibles = [(time, vertex, released, rate, activeValves, []),...]
    '''
    retval = dict()
    allValvesOpen = -1
    maxMinPotential = -1

    for state in possibles:
        newState, newVals = None, None
        time, vs, released, rate, A, P = state
        v1, v2 = vs
        newReleased = released + rate
        newTime = time + 1

        minPotential = newReleased + (numRemain*rate)
        maxPotential = newReleased + (numRemain*maxRoundReleased)
        maxMinPotential = max([minPotential, maxMinPotential])
        if maxPotential < maxMinPotential:
            #logging.info("cur {} minP {} maxP {} maxMinP {} state {}".format(
            #    newReleased, minPotential, maxPotential, maxMinPotential, state))
            #logging.info("Would bail")
            continue


        if newReleased < allValvesOpen:  
            continue
        
        if all_valves_open(A):
            #logging.info("allValvesOpen")
            newState = tuple([newTime] + vs + [A[x] for x in sorted(A.keys())])
            newVals = [newTime, vs, newReleased, rate, A.copy(), P.copy() + ['Pass','Pass']]
            prevVals = retval.get(newState, None)
            if prevVals == None or newReleased > prevVals[2]:
                retval[newState] = newVals
                allValvesOpen = max([allValvesOpen, newReleased])
                #logging.info("setting allValvesOpen")
            continue

            
        for action1 in ['open'] + V[v1].N:
            newV1 = v1
            Acopy = A.copy()
            newRate = rate

            if action1 == 'open' and Acopy.get(v1,True) == False: # and V[v1].rate > 0:
                Acopy[v1] = True
                newRate += V[v1].rate

                if all_valves_open(Acopy):
                    #logging.info("Done")
                    newState = tuple([newTime] + vs + [A[x] for x in sorted(A.keys())])
                    newVals = [newTime, vs, newReleased, rate, Acopy.copy(), P.copy() + ['open','pass']]
                    prevVals = retval.get(newState, None)
                    if prevVals == None or newReleased > prevVals[2]:
                        retval[newState] = newVals
                    continue
                
            elif action1 not in ['open']:
                newV1 = action1
            else:  # open when already open/rate==0
                continue  #skip it
                
            for action2 in ['open'] + V[v2].N:
                newState, newVals, newNewRate = None, None, newRate
                newP = P.copy() + [[action1, action2]]
                newP = []
            
                if action2 == 'open' and Acopy.get(v2,True) == False: # == False and V[v2].rate > 0:
                    Acopycopy = Acopy.copy()
                    Acopycopy[v2] = True
                    newNewRate += V[v2].rate
                    newState = tuple([newTime] + \
                                     sorted([newV1, v2]) + \
                                     [Acopycopy[x] for x in sorted(Acopycopy.keys())])
                    newVals = [newTime, sorted([newV1,v2]), newReleased, newNewRate, Acopycopy.copy(), newP]
                    
                elif action2 not in ['open']:
                    newState = tuple([newTime] + \
                                     sorted([newV1, action2]) + \
                                     [Acopy[x] for x in sorted(Acopy.keys())])
                    newVals = [newTime, sorted([newV1,action2]), newReleased, newNewRate, Acopy.copy(), newP] 

                if newState != None:
                    prevVals = retval.get(newState, None)
                    if prevVals == None:
                        #logging.info("First Time Encountered {}".format(newState,))
                        retval[newState] = newVals
                    elif newReleased > prevVals[2]:
                        # Previous state arrived here but pressure released was less
                        #logging.info("Better Than Path {}".format(prevVals,))
                        retval[newState] = newVals

    return retval.values()



maxRoundReleased = sum([v.rate for v in vertices.values()])

states = [[0, 'AA', 0, 0, valvesActive, []]]
numMoves = 30
for move in range(0, numMoves):
    states = make_next_moves(vertices, states, numMoves-move-1, maxRoundReleased)
    logging.info("After move {}; num states {}".format(move+1, len(states)))
partA = max([s[2] for s in states])

logging.info("16a. {} pressure released by best path.".format(partA,))



states2 = [[0, ['AA', 'AA'], 0, 0, valvesActive, []]]
numMoves = 26
for move in range(0, numMoves):
    states2 = make_next_moves2(vertices, states2, numMoves-move-1, maxRoundReleased)
    logging.info("After move {}; num states {}".format(move+1, len(states2)))
partB = max([s[2] for s in states2])


logging.info("16a. {} pressure released by best path.".format(partB,))


# Day 17:  Pyroclastic Flow

## 17a.  How many units tall will the tower of rocks be after 2022 rocks have stopped falling?

## 17b.  How tall will the tower be after 1000000000000 rocks have stopped?

In [38]:
# Efficient storage.  Periodicity.  Scaling.  Polyominoes.  Modular Arithmetic.

filename = 'prob17input.txt'
lines = get_string_input(filename)
wind = lines[0]
#wind = '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'

# the pieces
dash =  [[2,4],[3,4],[4,4],[5,4]]
plus =  [[3,4],[2,5],[3,5],[4,5],[3,6]]
el =    [[2,4],[3,4],[4,4],[4,5],[4,6]]
line =  [[2,4],[2,5],[2,6],[2,7]]
block = [[2,4],[3,4],[2,5],[3,5]]
pieces = [dash, plus, el, line, block]


def move_piece_down(rock):
    return [[x,y-1] for [x,y] in rock]

def move_piece_left(rock):
    return [[x-1,y] for [x,y] in rock]

def move_piece_right(rock):
    return [[x+1,y] for [x,y] in rock]

def is_blocked(rock, blocked):
    retval = any([tuple(r) in blocked for r in rock])
    if retval == False:  
        retval = any([r[0] < 0 for r in rock])
    if retval == False:  
        retval = any([r[0] > 6 for r in rock])
    if retval == False:
        retval = any([r[1] < 0 for r in rock])
    return retval


def check_for_repeats(sequence, pLen):
    ''' Check end of sequence to see if a subsequence is repeated
         having length modulo pLen.  If so, return the length of
         the repeated subsequence.
        pLen = length of pieces cycle (i.e. 5)
    '''
    retval = None
    n = len(sequence)
    # Gradually increase seqLen by pLen
    for subLen in range(pLen, n//2, pLen):
        # Check [n - 2*subLen, n-subLen] == [n - subLen, n]
        sameSeq = False
        for j in range(n - 2 * subLen, n - subLen):
            sameSeq = True
            if sequence[j] != sequence[j+subLen]:
                sameSeq = False
                break

        if sameSeq == True:  # repeated subsequence found!
            retval = subLen
            break
    return retval


# PartA and Part B

for numPieces in [2022, 1000000000000]:
    wLen = len(wind)
    pLen = len(pieces)
    wIndex = 0
    blocked = set()
    blockedHeight = -1
    windIndexes = []
    blockedHeights = []
    logging.info("numPieces = {}, wLen = {}".format(numPieces, wLen))

    for r in range(numPieces):
        windIndexes.append(wIndex)
        blockedHeights.append(blockedHeight)
    
        # check for repeats in sequences
        if r % pLen == 0:
            seqLen = check_for_repeats(windIndexes, pLen)
        
            if seqLen != None:
                # Sequence repeats, but is height change the same for both subsequences the same?
                heightGain1 = blockedHeights[-1] - blockedHeights[(-1*seqLen)-1]
                heightGain2 = blockedHeights[(-1*seqLen)-1] - blockedHeights[(-2*seqLen)-1]
                if heightGain1 == heightGain2:
                    # The repeat has been found.
                    break
    
        # Still here? Then no repeats found yet.  
        # Create rock and drop according to instructions
        rock = [[x,y+blockedHeight] for [x,y] in pieces[r % pLen]]
        falling = True
        while falling:
            newRock = None

            # Let wind push rock
            if wind[wIndex] == '>':    
                newRock = move_piece_right(rock)
            elif wind[wIndex] == '<':  
                newRock = move_piece_left(rock)
            wIndex = (wIndex + 1) % wLen
            if not is_blocked(newRock, blocked):
                # If wind movement moves rock into blocked square, reset
                rock = newRock
        
            # Let gravity pull rock
            newRock = move_piece_down(rock)
            if not is_blocked(newRock, blocked):
                rock = newRock
            else:
                # Rock cannot fall, becomes part of blocked.
                falling = False
                for r in rock:
                    blocked.add(tuple(r))
                blockedHeight = max([y for [x,y] in rock] + [blockedHeight])

        # Finished falling
    # Finished num pieces (sort of)

    if numPieces == 2022 and wLen != 40:
        partA = blockedHeight+1   # partA for should have nonzero seqLen
    elif seqLen != None:
        # A repeat of the windIndexes has been found.  Use it to skip ahead.
        relBase = blockedHeights[-seqLen-1] # height at end of first cycle
        relHeights = [x - relBase for x in blockedHeights[-seqLen:]] # heights gained from relBase
        numRemainPieces = numPieces - r  
        firstHeight = blockedHeights[-1]
        numRemainCycles = numRemainPieces//seqLen  # num full cycles that remain
        remainderCycle = numRemainPieces % seqLen # length of last partial cycle
        #newHeight = (curHeight) + (height for full cycles) + (zero based offset)
        partB = firstHeight + (numRemainCycles*relHeights[-1]) + 1
        if remainderCycle > 0:
            # + (height for last partial cycle)
            partB += relHeights[remainderCycle-1]  
        if numPieces == 2022:
            partA = partB
            
logging.info("17a. Max Height = {}".format(partA,))
logging.info("17b. Max Height = {}".format(partB,))

2023-01-24 09:40:14 INFO: Got 1 lines from prob17input.txt.
2023-01-24 09:40:14 INFO: numPieces = 2022, wLen = 10091
2023-01-24 09:40:14 INFO: numPieces = 1000000000000, wLen = 10091
2023-01-24 09:40:14 INFO: 17a. Max Height = 3092
2023-01-24 09:40:14 INFO: 17b. Max Height = 1528323699442


# Day 18:  Boiling Boulders

## 18a.  What is the surface area of your scanned lava droplet?

## 18b.  What is the external surface area of your scanned lava droplet?

In [39]:
# Vector addition.  Creative topology.

filename = 'prob18input.txt'
lines = get_string_input(filename)
#lines = ['2,2,2','1,2,2','3,2,2','2,1,2','2,3,2','2,2,1','2,2,3',
#         '2,2,4','2,2,6','1,2,5','3,2,5','2,1,5','2,3,5']

blocks = set()
for l in lines:
    blocks.add(tuple([int(x) for x in l.split(',')]))

neighbors = [[0,0,1],[0,0,-1],[0,1,0],[0,-1,0],[1,0,0],[-1,0,0]]

# Part A
partA = 0
for b in blocks:
    for n in neighbors:
        if tuple([x+y for x,y in zip(b,n)]) not in blocks:
            partA += 1

# Part B
# Construct set of external cubes
vMin = min([min(b) for b in blocks])
vMax = max([max(b) for b in blocks])

# Create a set of external squares.  Note that the box
#  [vMin-1, vMax+1] x [vMin-1, vMax+1] x [vMin-1, vMax+1] fully
#  contains all b in blocks and the boundary contains no b in blocks
#  Grow external set from (vMin-1,vMin-1,vMin-1) to get all external
#  blocks.
external = set()
toConsider = [(vMin-1,vMin-1,vMin-1)]
while len(toConsider) > 0:
    b = toConsider.pop()
    external.add(b)
    for n in neighbors:
        bNew = tuple([x+y for x,y in zip(b,n)])
        if min(bNew) >= vMin-1 and \
           max(bNew) <= vMax+1 and \
           bNew not in blocks and \
           bNew not in external:
            toConsider.append(bNew)

partB = 0
for b in blocks:
    for n in neighbors:
        if tuple([x+y for x,y in zip(b,n)]) in external:
            partB += 1

logging.info('18a. Surface Area: {}'.format(partA,))
logging.info('18b. External Surface Area: {}'.format(partB,))


2023-01-24 09:40:17 INFO: Got 2192 lines from prob18input.txt.
2023-01-24 09:40:17 INFO: 18a. Surface Area: 3412
2023-01-24 09:40:17 INFO: 18b. External Surface Area: 2018


# Day 19:  Not Enough Minerals

## 19a.  What do you get if you add up the quality level of all of the blueprints in your list?

## 19b.  What do you get if you multiply these numbers together?

In [40]:
# Code takes about a minute to finish.

# Basic algorithm is to have a queue of states and examine each state one at a
# time.  When examining state, consider the next robot to be purchased.  If
# there is adequate time to make that purchase, gain the required resources,
# make the purchase, and then create a new state with that purchase and add it
# to the queue.  Also add a state where no additional purchases are made.
# Continue examining states until the queue is empty.  Keep note of the largest
# number of geodes found while examining the states.

filename = 'prob19input.txt'
lines = get_string_input(filename)
'''lines = ['Blueprint 1:  Each ore robot costs 4 ore.  Each clay robot costs 2 ore.\
Each obsidian robot costs 3 ore and 14 clay.  Each geode robot costs 2 ore and 7 obsidian.',
         'Blueprint 2:  Each ore robot costs 2 ore.  Each clay robot costs 3 ore.\
         Each obsidian robot costs 3 ore and 8 clay.  Each geode robot costs 3 ore and 12 obsidian.']
'''
class Blueprint:
    def __init__(self, stats):
        self.bid = int(stats[0])
        self.ore = [0,0,0,int(stats[1])]  # costs for [-, obsidian, clay, ore]
        self.cly = [0,0,0,int(stats[2])]  #  same
        self.obs = [0,0,int(stats[4]),int(stats[3])]  # same
        self.geo = [0,int(stats[6]),0,int(stats[5])]  # same

    def adequate_ore(self, resources):
        return (self.ore[3] <= resources[3])

    def adequate_cly(self, resources):
        return (self.cly[3] <= resources[3])

    def adequate_obs(self, resources):
        return ((self.obs[3] <= resources[3]) and (self.obs[2] <= resources[2]))

    def adequate_geo(self, resources):
        return ((self.geo[3] <= resources[3]) and (self.geo[1] <= resources[1]))
    
    def display(self):
        logging.info("Blueprint {}: {} {} {} {}".format(self.bid, self.geo, self.obs, self.cly, self.ore))
    
        
class State:
    def __init__(self):
        self.resources = [0,0,0,0]
        self.robots = [0,0,0,0]
        self.depth = 0
        self.minPotential = 0
        self.maxPotential = 0

    def copy_state(self):
        retval = State()
        retval.resources = [x for x in self.resources]
        retval.robots = [x for x in self.robots]
        retval.depth = self.depth
        return retval
        
    def display(self):
        logging.info("depth = {} potential = {}/{} res = {}, rob = {}".format(
            self.depth, self.minPotential, self.maxPotential, self.resources, self.robots))

    def gain_resources(self):
        for i in range(len(self.resources)):
            self.resources[i] += self.robots[i]

    def gain_robot(self, robotType):
        if   robotType == 'ore':  self.robots[3] += 1
        elif robotType == 'cly':  self.robots[2] += 1
        elif robotType == 'obs':  self.robots[1] += 1
        elif robotType == 'geo':  self.robots[0] += 1
            
    def rounds_to_purchase(self, bp, robotType):
        ''' Calculate and return the number of rounds needed to gain the
            resources required to purchase robot of robotType.
            Return None if not possible in current state
             (e.g. robotType = obs and state has no cly)
        '''
        retval = None
        if robotType == 'ore': 
            retval = math.ceil((bp.ore[3] - self.resources[3]) / self.robots[3])
        elif robotType == 'cly': 
            retval = math.ceil((bp.cly[3] - self.resources[3]) / self.robots[3])
        elif robotType == 'obs' and self.robots[2] > 0: 
            retval = math.ceil((bp.obs[3] - self.resources[3]) / self.robots[3])
            retval = max([retval, math.ceil((bp.obs[2] - self.resources[2]) / self.robots[2])])
        elif robotType == 'geo' and self.robots[1] > 0: 
            retval = math.ceil((bp.geo[3] - self.resources[3]) / self.robots[3])
            retval = max([retval, math.ceil((bp.geo[1] - self.resources[1]) / self.robots[1])])
        if retval != None:
            retval = max([retval,0])
        return retval
    
    def purchase_robot(self, bp, robotType):
        # robot purchased but not acquired
        if robotType == 'ore':
            self.resources[3] -= bp.ore[3]
        elif robotType == 'cly':
            self.resources[3] -= bp.cly[3]
        elif robotType == 'obs':
            self.resources[3] -= bp.obs[3]
            self.resources[2] -= bp.obs[2]
        elif robotType == 'geo':
            self.resources[3] -= bp.geo[3]
            self.resources[1] -= bp.geo[1]
            
            
    def evaluate_potential2(self, maxDepth, bp):
        ''' Non-trivial heuristic to evaluate minimal and maximal bounds
            for current state.  
            For minimal potential, determine number of geodes if no more
            purchases are made.
            For maximal potential, "purchase" geo robot if possible but
            only pay obsidian.  If not possible, "purchase" obs if possible
            but only pay cly; also acquire ore and cly without paying.
        '''
        k = maxDepth - self.depth  # remaining rounds
        self.minPotential = self.resources[0] + (self.robots[0]*k)
        
        robots = [x for x in self.robots]
        resources = [x for x in self.resources]
        # foreach subsequent round
        for i in range(k):
            robotsToAdd = ['ore','cly']
            if bp.obs[2] <= resources[2]:
                robotsToAdd.append('obs')
            if bp.geo[1] <= resources[1]:
                robotsToAdd = ['geo']

            # gain resources
            for j in range(4):
                resources[j] += robots[j]

            # purchase robot (just pay obs or cly)
            for r in robotsToAdd:
                if r == 'ore':
                    robots[3] += 1
                elif r == 'cly':
                    robots[2] += 1
                elif r == 'obs':
                    robots[1] += 1
                    resources[2] -= bp.obs[2]
                elif r == 'geo':
                    robots[0] += 1
                    resources[1] -= bp.geo[1]

        self.maxPotential = resources[0]
        
        
line_re = re.compile('^Blueprint ([\d]+): .*? ([\d]+) ore.*? ([\d]+) ore.*? ([\d]+) \
ore and ([\d]+) clay.*? ([\d]+) ore and ([\d]+) obsidian.$')

blueprints = []
for l in lines:
    m1 = line_re.match(l)
    if m1 != None:
        blueprints.append(Blueprint(m1.groups(1)))


def evaluate_state(bp, curState, maxDepth):
    ''' For the currrent state, attempt to buy each robot.  If
        possible, create a new state, gain resources to required,
        purchase robot, and add that new state to the queue.
        Also add the state of no purchase to the queue.
        For each new state, evaluate its minimal and maximum
        potential.
    '''
    retval = []  # New states
    maxGeodes = 0
    if curState.depth >= maxDepth:
        return retval
    
    # purchase robots
    for robotType in ['geo','obs','cly','ore']:
        roundsToPurchase = curState.rounds_to_purchase(bp, robotType)
        if roundsToPurchase != None:
            if curState.depth + roundsToPurchase + 1 >= maxDepth:
                # roundsToPurchase plus round of purchase
                continue
            newState = curState.copy_state()
            for j in range(roundsToPurchase):
                newState.depth += 1
                newState.gain_resources()
            # newState now has enough resources to purchase robot
            newState.purchase_robot(bp, robotType)
            newState.depth += 1
            newState.gain_resources()
            newState.gain_robot(robotType)
            newState.evaluate_potential2(maxDepth, bp)
            retval.append(newState)            

    # Consider no purchase
    if curState.robots[0] > 0:
        newState = curState.copy_state()
        #for j in range(maxDepth - curState.depth -1):
        while newState.depth < maxDepth:
            newState.depth += 1
            newState.gain_resources()
        newState.evaluate_potential2(maxDepth, bp)
        retval.append(newState)

    return retval



#partA and partB
for maxDepth in [24, 32]:  
    qualityLevels, allMaxGeodes = [], []  # required for part A / part B
    
    for numBp, bp in enumerate(blueprints):
        if numBp >= 3 and maxDepth == 32:
            break  # break condition for part B
        s = State()      # create initial state
        s.robots[3] += 1 # kickoff with single ore robot
        possibles = [s]  # add initial state to list of states to be evaluated
        maxGeodes, minPotential, maxPossibles, numEval = 0, 0, 0, 0

        while len(possibles) > 0:
            s = possibles.pop(0)  # pop first element
            numEval += 1          # keep a count of states evaluated
            if s.maxPotential < minPotential:
                # s-state maximum potential does not exceed some other
                # state's minimum potential... discard s; not worth pursuing
                continue

            # save the largest minimum potential, maxGeoges will be at least this value
            minPotential = max([minPotential, s.minPotential])
            maxGeodes = max([maxGeodes, s.resources[0]])

            # Get new states from state s and add to queue if there is potential
            for ns in evaluate_state(bp, s, maxDepth):
                if ns.maxPotential >= minPotential and ns.maxPotential > 0:
                    possibles.append(ns)

            # Not sure how queue should be sorted, but this method is pretty fast.
            possibles = sorted(possibles, key=lambda x: [x.maxPotential, x.robots, x.depth], reverse=True)
            maxPossibles = max([maxPossibles, len(possibles)])  # keep track of largest queue size
    
        qLevel = (numBp+1) * maxGeodes  # for part A
        qualityLevels.append(qLevel)    # for part A 
        allMaxGeodes.append(maxGeodes)  # for part B
        # Finished with Blueprint, output stats...
        logging.info("Blueprint {}: maxGeodes = {}, qLevel = {}, maxPossibles = {}, numEval = {}".format(
            numBp+1, maxGeodes, qLevel, maxPossibles, numEval))

    if maxDepth == 24:
        logging.info("19a. Sum Quality Levels = {}".format(sum(qualityLevels),))
    elif maxDepth == 32:
        logging.info("19b. Product of Max Geodes = {}".format(np.prod(allMaxGeodes),))

        

2023-01-24 09:40:20 INFO: Got 30 lines from prob19input.txt.
2023-01-24 09:40:20 INFO: Blueprint 1: maxGeodes = 4, qLevel = 4, maxPossibles = 28, numEval = 51
2023-01-24 09:40:20 INFO: Blueprint 2: maxGeodes = 0, qLevel = 0, maxPossibles = 2, numEval = 5
2023-01-24 09:40:20 INFO: Blueprint 3: maxGeodes = 1, qLevel = 3, maxPossibles = 23, numEval = 1462
2023-01-24 09:40:20 INFO: Blueprint 4: maxGeodes = 3, qLevel = 12, maxPossibles = 78, numEval = 196
2023-01-24 09:40:20 INFO: Blueprint 5: maxGeodes = 9, qLevel = 45, maxPossibles = 85, numEval = 140
2023-01-24 09:40:20 INFO: Blueprint 6: maxGeodes = 1, qLevel = 6, maxPossibles = 14, numEval = 109
2023-01-24 09:40:20 INFO: Blueprint 7: maxGeodes = 0, qLevel = 0, maxPossibles = 4, numEval = 13
2023-01-24 09:40:21 INFO: Blueprint 8: maxGeodes = 3, qLevel = 24, maxPossibles = 48, numEval = 16332
2023-01-24 09:40:21 INFO: Blueprint 9: maxGeodes = 3, qLevel = 27, maxPossibles = 42, numEval = 5398
2023-01-24 09:40:21 INFO: Blueprint 10: maxGeo

# 20.  Grove Positioning System

## 20a.  What is the sum of the three numbers that form the grove coordinates?

## 20b.  What is the sum of the three numbers that form the grove coordinates?

In [41]:
# Code takes about 2 minutes to run.  Could probably be faster
# with pointers.

# Pointers.  Classes.  Linked Lists.

filename = 'prob20input.txt'
nums = get_integer_input(filename)
#nums = [1,2,-3,3,-2,0,4]

class Number:
    def __init__(self, val, index):
        self.val = val
        self.index = index
        self.moved = 0

# part A
# Basic algorithm: Examine first item in list.  Move to new
#  location if it has not been previously moved.  Otherwise
#  just put it at the end of the list.  Continue until all
#  elements in the list have been moved.
myArray = [Number(n, i) for i, n in enumerate(nums)]
numToMove = len(myArray)
numMoved = 0
while numMoved < numToMove:
    n = myArray.pop(0)
    if n.moved == 0:   # if value has not been moved
        n.moved += 1   #  this is the next element to be moved
        newIndex = n.val % len(myArray)  # works for all cases
        myArray = myArray[:newIndex] + [n] + myArray[newIndex:]
        numMoved += 1
    else:   # value has already been moved
        myArray.append(n)  # kick value at the end of the list
        
final = [n.val for n in myArray]
zeroIndexA = final.index(0)
partA = sum([final[(zeroIndexA+k) % numToMove] for k in [1000,2000,3000]])


# Part B
# Basic algorithm: Similar to part A... For each round, 
#  examine first item in list.  Move to new location if it 
#  is the next item to be moved.  Otherwise just put it at
#  the end of the list.  Continue round until all elements
#  in the list have been moved.  Note: After the first 
#  round, the elements are essentially in random order so
#  the inner loop has to churn through the list a lot more
#  until the correct item to move is encountered.  

decryptKey = 811589153
myArray = [Number(n*decryptKey, i) for i, n in enumerate(nums)]
numToMove = len(myArray)
numMoved, count = 0, 0
rounds = 10
for r in range(rounds):
    logging.info("Beginning round {} {} {}".format(r, numMoved, count))
    numMoved = 0
    while numMoved < numToMove:
        count += 1  # Keep track of number of examined items
        n = myArray.pop(0)
        if n.moved == r and n.index == numMoved: # is next item to be moved?
            n.moved += 1
            newIndex = n.val % len(myArray)
            myArray = myArray[:newIndex] + [n] + myArray[newIndex:]
            numMoved += 1
        else:
            myArray.append(n)
        
final = [n.val for n in myArray]
zeroIndexB = final.index(0)
partB = sum([final[(zeroIndexB+k) % numToMove] for k in [1000,2000,3000]])


logging.info("20a. Sum of grove coordinates: {}".format(partA,))
logging.info("20b. Sum of grove coordinates: {}".format(partB,))
    

2023-01-24 09:40:36 INFO: Got 5000 integers from prob20input.txt.
2023-01-24 09:40:36 INFO: Beginning round 0 0 0
2023-01-24 09:40:36 INFO: Beginning round 1 5000 8584
2023-01-24 09:40:50 INFO: Beginning round 2 5000 12565182
2023-01-24 09:41:03 INFO: Beginning round 3 5000 25036708
2023-01-24 09:41:17 INFO: Beginning round 4 5000 37458344
2023-01-24 09:41:30 INFO: Beginning round 5 5000 50014895
2023-01-24 09:41:44 INFO: Beginning round 6 5000 62486447
2023-01-24 09:41:57 INFO: Beginning round 7 5000 74878059
2023-01-24 09:42:10 INFO: Beginning round 8 5000 87364594
2023-01-24 09:42:23 INFO: Beginning round 9 5000 99911134
2023-01-24 09:42:36 INFO: 20a. Sum of grove coordinates: 8302
2023-01-24 09:42:36 INFO: 20b. Sum of grove coordinates: 656575624777


# 21.  Monkey Math

## 21a.  What number will the monkey named root yell?

## 21b.  What number do you yell to pass root's equality test?

In [42]:
# Recursion.  Binary Trees.
# Part II.  Weird recursion to find value desired and then
#  push that value down to solve for child value.

filename = 'prob21input.txt'
lines = get_string_input(filename)
'''lines = ['root: pppw + sjmn','dbpl: 5','cczh: sllz + lgvd','zczc: 2',
         'ptdq: humn - dvpt','dvpt: 3','lfqf: 4','humn: 5','ljgn: 2',
         'sjmn: drzm * dbpl','sllz: 4','pppw: cczh / lfqf',
         'lgvd: ljgn * ptdq','drzm: hmdt - zczc','hmdt: 32']
'''
line_re = re.compile('^(.*?): (.*?) ([\+\-\*\/]) (.*?)$')
numline_re = re.compile('^(.*?): (\d+)$')

class Monkey:
    def __init__(self, val, parentMonkey, leftMonkey, rightMonkey, operation):
        self.val = val
        self.name = parentMonkey
        self.lMonkey = leftMonkey
        self.rMonkey = rightMonkey
        self.operation = operation

    def display(self):
        if self.operation == None:
            logging.info("Value Monkey {}: {}".format(self.name, self.val))
        else:
            val = self.val
            if val == None:
                val = '?'
            logging.info("Op Monkey {}: {} {} {} = {}".format(
                self.name, self.lMonkey, self.operation, self.rMonkey, self.val))

# Parse input
monkeys = dict()
for l in lines:
    m1 = line_re.match(l)
    m2 = numline_re.match(l)
    if m1 != None:
        monkey, leftMonkey, operation, rightMonkey = m1.groups(1)
        monkeys[monkey] = Monkey(monkey, None, leftMonkey, rightMonkey, operation)
    elif m2 != None:
        monkey, val = m2.groups(1)
        monkeys[monkey] = Monkey(int(val), monkey, None, None, None)
    else:
        logging.error("Failed to parse '{}'".format(l,))

        
        
def evaluate_monkey(monkeys, name):
    ''' Recursive function.  Evaluate left child, evaluate right child,
         then evaluate current monkey and return its value.
    '''
    retval = None
    if monkeys[name].operation == None:
        retval = monkeys[name].val
    else:
        leftVal = evaluate_monkey(monkeys, monkeys[name].lMonkey)
        rightVal = evaluate_monkey(monkeys, monkeys[name].rMonkey)
        if monkeys[name].operation == '/':
            if rightVal == 0 or leftVal % rightVal != 0:
                logging.error("Trying to evaluate {}/{}".format(leftVal, rightVal))
        retval = int(eval("{} {} {}".format(leftVal, monkeys[name].operation, rightVal)))
    return retval


def evaluate_monkey2(monkeys, name, target):
    ''' Strange recursive function.
        Basic premise:  evaluate like the previous function
         with some alterations.
         1) If the target has not been set and not the root:
           1a) If the 'humn' value, just return 'humn'
           1b) If one of the left or right children return 'humn', just return 'humn'
           1c) Otherwise return as normal
         2) If the target has been set and not the root:
           2a) If the 'humn' value, return the target
           2b) If child returns 'humn', evaluate target for child and give to child
                Return value child returns, which should be humn target
         3) If root, evaluate both children.  Give value of non 'humn' child as
             target to 'humn' child.  Return value of what child returns, which
             should be humn target.
        This function could probably be better.
    '''
    retval = None
    if monkeys[name].operation == None:
        retval = monkeys[name].val
        if name == 'humn' and target != None:
            logging.info("Target for humn = {}".format(target,))
            retval = target
        elif name == 'humn':
            retval = 'humn'

    else:
        leftVal = evaluate_monkey2(monkeys, monkeys[name].lMonkey, None)
        rightVal = evaluate_monkey2(monkeys, monkeys[name].rMonkey, None)
        if rightVal == 'humn':
            retval = 'humn'
            if name == 'root':
                retval = evaluate_monkey2(monkeys, monkeys[name].rMonkey, leftVal)

            elif target != None:
                if monkeys[name].operation == '+':
                    newTarget = target - leftVal
                elif monkeys[name].operation == '-':
                    newTarget = leftVal - target
                elif monkeys[name].operation == '*':
                    if leftVal == 0 or target % leftVal != 0:
                        logging.error("Trying to evaluate {}/{}".format(target, leftVal))
                    newTarget = int(target/leftVal)
                elif monkeys[name].operation == '/':
                    if target == 0 or leftVal % target != 0:
                        logging.error("Trying to evaluate {}/{}".format(leftVal, target))
                    newTarget = int(leftVal/target)
                retval = evaluate_monkey2(monkeys, monkeys[name].rMonkey, newTarget)
                
        elif leftVal == 'humn':
            retval = 'humn'
            if name == 'root':
                retval = evaluate_monkey2(monkeys, monkeys[name].lMonkey, rightVal)

            elif target != None:
                if monkeys[name].operation == '+':
                    newTarget = target - rightVal
                elif monkeys[name].operation == '-':
                    newTarget = target + rightVal
                elif monkeys[name].operation == '*':
                    if rightVal == 0 or target % rightVal != 0:
                        logging.error("Trying to evaluate {}/{}".format(target, rightVal))
                    newTarget = int(target/rightVal)
                elif monkeys[name].operation == '/':
                    newTarget = target * rightVal
                retval = evaluate_monkey2(monkeys, monkeys[name].lMonkey, newTarget)

        else:  #leftVal != humn and rightVal != humn
            if monkeys[name].operation == '/':
                if rightVal == 0 or leftVal % rightVal != 0:
                    logging.error("Trying to evaluate {}/{}".format(leftVal, rightVal))
            retval = int(eval("{} {} {}".format(leftVal, monkeys[name].operation, rightVal)))

    return retval


partA = evaluate_monkey(monkeys, 'root')
partB = evaluate_monkey2(monkeys, 'root', None)
    
logging.info("21a. Monkey Number: {}".format(partA,))
logging.info("21b. Human Number: {}".format(partB,))


2023-01-24 09:42:45 INFO: Got 2361 lines from prob21input.txt.
2023-01-24 09:42:45 INFO: Target for humn = 3352886133831
2023-01-24 09:42:45 INFO: 21a. Monkey Number: 158661812617812
2023-01-24 09:42:45 INFO: 21b. Human Number: 3352886133831


# 22.  Monkey Map

## 22a.  What is the final password?

## 22b.  What is the final password?

In [43]:
# This code is messy.  The hard part was writing the code to transition
# from one face to another for part B.  Once that is hard coded the
# execution is straightforward.  Identifying the movement from one face
# to another is not easy.

filename = 'prob22input.txt'
lines = get_string_input(filename)
testlines = ['        ...#',
         '        .#..',
         '        #...',
         '        ....',
         '...#.......#',
         '........#...',
         '..#....#....',
         '..........#.',
         '        ...#....',
         '        .....#..',
         '        .#......',
         '        ......#.',
         '',
         '10R5L5R10L4R5L5']


def move_horizontal(pos, theMap, leftOrRight):
    ''' pos = [r,c] with r increasing moving downward
        theMap[r][c] is in [' ', '.', '#']
        leftOrRight is in [-1,1] for left or right, resp
        This is movement for partA
    '''
    retval = pos
    r, c = pos
    cNext = c + leftOrRight
    while 1:
        if cNext < 0 :                 cNext = len(theMap[r])-1
        elif cNext >= len(theMap[r]):  cNext = 0

        if theMap[r][cNext] == ' ':
            cNext += leftOrRight
        elif theMap[r][cNext] == '.':
            retval = [r, cNext]
            break
        elif theMap[r][cNext] == '#':
            break
    return retval


def move_verticle(pos, theMap, upOrDown):
    ''' pos = [r,c] with r increasing moving downward
        theMap[r][c] is in [' ', '.', '#'] if it exists
        upOrDown is in [-1,1] for up or down, resp
        This is movement for partA
    '''
    retval = pos
    r, c = pos
    rNext = r + upOrDown
    while 1:
        if rNext < 0:               rNext = len(theMap) - 1
        elif rNext >= len(theMap):  rNext = 0

        if c >= len(theMap[rNext]):
            rNext += upOrDown
        elif theMap[rNext][c] == ' ':
            rNext += upOrDown
        elif theMap[rNext][c] == '.':
            retval = [rNext, c]
            break
        elif theMap[rNext][c] == '#':
            break
    return retval



def move_horizontal2_test(pos, theMap, leftOrRight):
    ''' pos = [r,c] with r increasing moving downward
        theMap[r][c] is in [' ', '.', '#']

            12     1
            3    234
           45      56
           6
    '''
    newDir = 0
    if leftOrRight == -1:
        newDir = 2
    retval = [newDir, pos]

    r, c = pos
    rNext, cNext = r, c + leftOrRight
    sideLen = 4

    if cNext < 0 and sideLen <= r < 2*sideLen:
        # Moving left off face 2, come up on face 6 
        cNext = 3*sideLen + (2*sideLen-r-1)
        rNext = 3*sideLen - 1 
        newDir = 3 # now facing up
    elif cNext == 2*sideLen-1 and 0 <= r < sideLen:
        # Moving left off face 1, come down on face 3
        cNext = sideLen + r
        rNext = sideLen 
        newDir = 1 # now facing down
    elif cNext == 2*sideLen-1 and 2*sideLen <= r < 3*sideLen:
        # Moving left off face 5, come up on face 3
        cNext = sideLen + (3*sideLen-1-r)
        rNext = 2*sideLen - 1 
        newDir = 3 # now facing up
    elif cNext == 3*sideLen and 0 <= r < sideLen:
        # Moving right off face 1, come left on face 6 
        cNext = 4*sideLen - 1
        rNext = 3*sideLen - 1 - r 
        newDir = 2 # now facing left
    elif cNext == 3*sideLen and sideLen <= r < 2*sideLen:
        #logging.info("Moving right off face 4, come down on face 6")
        cNext = 3*sideLen + (2*sideLen-1 - r)
        rNext = 2*sideLen 
        newDir = 1 # now facing down
    elif cNext == 4*sideLen and 2*sideLen <= r < 3*sideLen:
        # Moving right off face 6, come left on face 1 
        cNext = 3*sideLen-1
        rNext = 3*sideLen-1 - r 
        newDir = 2 # now facing left

    if theMap[rNext][cNext] == '.':
        retval = [newDir, [rNext, cNext]]
    elif theMap[rNext][cNext] == '#':
        pass  # let retval remain the default

    return retval


def move_verticle2_test(pos, theMap, upOrDown):
    ''' pos = [r,c] with r increasing moving downward
        theMap[r][c] is in [' ', '.', '#']

            12     1
            3    234
           45      56
           6
    '''
    newDir = 1
    if upOrDown == -1:
        newDir = 3
    retval = [newDir, pos]
    r, c = pos
    rNext, cNext = r + upOrDown, c
    sideLen = 4

    if rNext < 0 and 2*sideLen <= c < 3*sideLen:
        # Moving up off face 1, come down on face 2 
        cNext = 3*sideLen-1-c
        rNext = sideLen
        newDir = 1 # now facing down
    elif rNext == sideLen-1 and 0 <= c < sideLen:
        # Moving up off face 2, come down on face 1
        cNext = 3*sideLen-1-c
        rNext = 0
        newDir = 1 # now facing down
    elif rNext == sideLen-1 and sideLen <= c < 2*sideLen:
        # Moving up off face 3, come right on face 1
        cNext = 2*sideLen 
        rNext = c-sideLen
        newDir = 0 # now facing right
    elif rNext == 2*sideLen-1 and 3*sideLen <= c < 4*sideLen:
        # Moving up off face 6, come left on face 4
        cNext = 3*sideLen
        rNext = c-(2*sideLen)
        newDir = 2 # now facing left

    elif rNext == 2*sideLen and 0 <= c < sideLen:
        # Moving down off face 2, come up on face 5
        cNext = 3*sideLen-1 - c
        rNext = 3*sideLen-1
        newDir = 3 # now facing up
    elif rNext == 2*sideLen and sideLen <= c < 2*sideLen:
        # Moving down off face 3, come right on face 5
        cNext = 2*sideLen
        rNext = 3*sideLen-1 - (c-sideLen)
        newDir = 0 # now facing right
    elif rNext == 3*sideLen and 2*sideLen <= c < 3*sideLen:
        # Moving down off face 5, come up on face 2
        cNext = 3*sideLen-1 - c
        rNext = 2*sideLen-1
        newDir = 3 # now facing up
    elif rNext == 3*sideLen and 3*sideLen <= c < 4*sideLen:
        # Moving down off face 6, come right on face 2
        cNext = 0
        rNext = sideLen + 4*sideLen-1-c 
        newDir = 0 # now facing right

    if theMap[rNext][cNext] == '.':
        retval = [newDir, [rNext, cNext]]
    elif theMap[rNext][cNext] == '#':
        pass  # let retval remain the default

    return retval





def move_right(pos, theMap):
    ''' pos = [r,c] with r increasing moving downward    12
        theMap[r][c] is in [' ', '.', '#']               3
                                                        45
                                                        6
    '''
    newDir = 0
    retval = [newDir, pos]
    sideLen = 50
    r, c = pos
    rNext, cNext = r, c + 1

    if cNext == sideLen and 3*sideLen <= r < 4*sideLen:
        # Moving right off face 6, come up on face 5 
        newDir, rNext, cNext = 3, 3*sideLen - 1, r-(2*sideLen)
    elif cNext == 2*sideLen and sideLen <= r < 2*sideLen:
        # Moving right off face 3, come up on face 2
        newDir, rNext, cNext = 3, sideLen - 1, r+sideLen
    elif cNext == 2*sideLen and 2*sideLen <= r < 3*sideLen:
        # Moving right off face 5, come left on face 2
        newDir, rNext, cNext = 2, (3*sideLen-1) - r, 3*sideLen - 1
    elif cNext == 3*sideLen and 0 <= r < sideLen:
        # Moving right off face 2, come left on face 5
        newDir, rNext, cNext = 2, (3*sideLen-1) - r, 2*sideLen - 1

    if theMap[rNext][cNext] == '.':
        retval = [newDir, [rNext, cNext]]
    elif theMap[rNext][cNext] == '#':
        pass  # let retval remain the default
    
    return retval


def move_down(pos, theMap):
    ''' pos = [r,c] with r increasing moving downward    12
        theMap[r][c] is in [' ', '.', '#']               3
                                                        45
                                                        6
    '''
    newDir = 1
    retval = [newDir, pos]
    sideLen = 50
    r, c = pos
    rNext, cNext = r+1, c

    if rNext == sideLen and 2*sideLen <= c < 3*sideLen:
        # Moving down off face 2, come left on face 3
        newDir, rNext, cNext = 2, c - sideLen, 2*sideLen - 1
    elif rNext == 3*sideLen and sideLen <= c < 2*sideLen:
        # Moving down off face 5, come left on face 6
        newDir, rNext, cNext = 2, c + (2*sideLen), sideLen-1
    elif rNext == 4*sideLen and 0 <= c < sideLen:
        # Moving down off face 6, come down on face 2
        newDir, rNext, cNext = 1, 0, c + (2*sideLen)

    if theMap[rNext][cNext] == '.':
        retval = [newDir, [rNext, cNext]]
    elif theMap[rNext][cNext] == '#':
        pass  # let retval remain the default

    return retval


def move_left(pos, theMap):
    ''' pos = [r,c] with r increasing moving downward    12
        theMap[r][c] is in [' ', '.', '#']               3
                                                        45
                                                        6
    '''
    newDir = 2
    retval = [newDir, pos]
    sideLen = 50
    r, c = pos
    rNext, cNext = r, c -1

    if cNext < 0 and 2*sideLen <= r < 3*sideLen:
        # Moving left off face 4, come right on face 1 
        newDir, rNext, cNext = 0, 3*sideLen-1 - r, sideLen
    elif cNext < 0 and 3*sideLen <= r < 4*sideLen:
        # Moving left off face 6, come down on face 1 
        newDir, rNext, cNext = 1, 0, r - (2*sideLen)
    elif cNext == sideLen-1 and 0 <= r < sideLen:
        # Moving left off face 1, come right on face 4
        newDir, rNext, cNext = 0, 3*sideLen-1 - r, 0
    elif cNext == sideLen-1 and sideLen <= r < 2*sideLen:
        # Moving left off face 3, come down on face 4
        newDir, rNext, cNext = 1, 2*sideLen, r-sideLen

    if theMap[rNext][cNext] == '.':
        retval = [newDir, [rNext, cNext]]
    elif theMap[rNext][cNext] == '#':
        pass  # let retval remain the default

    return retval


def move_up(pos, theMap):
    ''' pos = [r,c] with r increasing moving downward    12
        theMap[r][c] is in [' ', '.', '#']               3
                                                        45
                                                        6
    '''
    newDir = 3
    retval = [newDir, pos]
    sideLen = 50
    r, c = pos
    rNext, cNext = r-1, c

    if rNext < 0 and sideLen <= c < 2*sideLen:
        # Moving up off face 1, come right on face 6
        newDir, rNext, cNext = 0, c + 2*sideLen, 0
    elif rNext < 0 and 2*sideLen <= c < 3*sideLen:
        # Moving up off face 2, come up on face 6
        newDir, rNext, cNext = 3, 4*sideLen - 1, c - (2*sideLen)
    elif rNext == 2*sideLen-1 and 0 <= c < sideLen:
        # Moving up off face 4, come right on face 3
        newDir, rNext, cNext = 0, c + sideLen, sideLen

    if theMap[rNext][cNext] == '.':
        retval = [newDir, [rNext, cNext]]
    # else hit a wall, leave as default
    return retval


def parse_input(myLines):
    course, theMap = [], []
    for l in myLines:
        if l == '':   break
        else:         theMap.append(l)

    lastLine = myLines[-1]
    while len(lastLine) > 0:
        m1 = re.match('^([RL]?)(\d+)(.*)$', lastLine)
        if m1 != None:
            course.append([m1.groups(1)[0], int(m1.groups(1)[1])])
            lastLine = m1.groups(1)[2]
        else:
            break

    return [course, theMap]


# Parse Input
test = False  # Set to true to run test
course, theMap = [], []
if test == True:
    course, theMap = parse_input(testlines)
else:
    course, theMap = parse_input(lines)
    
#PartA  (test is irrelevant)
directions = ['right','down','left','up']
d = 0
pos = move_horizontal([0,-1], theMap, 1)  # setup

for stepDir, stepCount in course:
    #logging.info("Beginning next step at {} facing {}".format(pos, directions[d]))
    if stepDir == 'R':    d = (d+1) % len(directions)
    elif stepDir == 'L':  d = (d-1) % len(directions)        
        
    for s in range(stepCount):
        if d == 0:
            pos = move_horizontal(pos, theMap, 1)
        elif d == 1:
            pos = move_verticle(pos, theMap, 1)
        elif d == 2:
            pos = move_horizontal(pos, theMap, -1)
        elif d == 3:
            pos = move_verticle(pos, theMap, -1)
        
logging.info("Finished at {} facing {}".format(pos, directions[d]))
pswdA = (1000*(pos[0]+1)) + (4*(pos[1]+1)) + d

# Part B
directions = ['right','down','left','up']
d = 0
pos = move_horizontal([0,1], theMap, 1)

for stepDir, stepCount in course:
    #logging.info("Beginning next step at {} facing {}".format(pos, directions[d]))
    if stepDir == 'R':    d = (d+1) % len(directions)
    elif stepDir == 'L':  d = (d-1) % len(directions)        
        
    if test == True:
        for s in range(stepCount):
            if d == 0:
                d, pos = move_horizontal2_test(pos, theMap, 1)
            elif d == 1:
                d, pos = move_verticle2_test(pos, theMap, 1)
            elif d == 2:
                d, pos = move_horizontal2_test(pos, theMap, -1)
            elif d == 3:
                d, pos = move_verticle2_test(pos, theMap, -1)
    else:
        for s in range(stepCount):
            if d == 0:
                d, pos = move_right(pos, theMap)
            elif d == 1:
                d, pos = move_down(pos, theMap)
            elif d == 2:
                d, pos = move_left(pos, theMap)
            elif d == 3:
                d, pos = move_up(pos, theMap)


logging.info("Finished at {} facing {}".format(pos, directions[d]))
pswdB = (1000*(pos[0]+1)) + (4*(pos[1]+1)) + d


logging.info("Test = {}".format(test,))
logging.info("22a.  Password = {}".format(pswdA,))
logging.info("22a.  Password = {}".format(pswdB,))

2023-01-24 09:42:49 INFO: Got 202 lines from prob22input.txt.
2023-01-24 09:42:49 INFO: Finished at [116, 12] facing left
2023-01-24 09:42:49 INFO: Finished at [161, 23] facing right
2023-01-24 09:42:49 INFO: Test = False
2023-01-24 09:42:49 INFO: 22a.  Password = 117054
2023-01-24 09:42:49 INFO: 22a.  Password = 162096


# 23.  Unstable Diffusion

## 23a.  How many empty ground tiles does that rectangle contain?

## 23b.  What is the number of the first round where no Elf moves?

In [44]:
# Choosing the right structure is helpful.  Growing grids scare me.

filename = 'prob23input.txt'
lines = get_string_input(filename)
#lines = ['.....','..##.','..#..','.....','..##.','.....']
'''lines = ['..............','..............','.......#......','.....###.#....',
'...#...#.#....','....#...##....','...#.###......','...##.#.##....',
'....#..#......','..............','..............','..............']
'''

def can_stay_put(x, y, elves):
    retval = True
    for p in [(x-1,y+1),(x,y+1),(x+1,y+1),(x-1,y),(x+1,y),(x-1,y-1),(x,y-1),(x+1,y-1)]:
        if p in elves:
            retval = False
            break
    return retval

def can_move_north(x, y, elves):
    return all([p not in elves for p in [(x-1,y+1),(x,y+1),(x+1,y+1)]])
    
def can_move_south(x, y, elves):
    return all([p not in elves for p in [(x-1,y-1),(x,y-1),(x+1,y-1)]])

def can_move_west(x, y, elves):
    return all([p not in elves for p in [(x-1,y-1),(x-1,y),(x-1,y+1)]])

def can_move_east(x, y, elves):
    return all([p not in elves for p in [(x+1,y-1),(x+1,y),(x+1,y+1)]])

# Parse Input
n = len(lines)
elves = set()
for i in range(n):
    for j in range(len(lines[0])):
        if lines[n-i-1][j] == '#':
            elves.add((j,i))   # treating as [x,y] coords

movements = ['north','south','west','east']
rounds = 0
while 1:
    rounds += 1
    moved, proposedMoves, counts = False, dict(), dict()
    
    for elf in elves:
        newPos = (elf[0], elf[1])
        if not can_stay_put(elf[0], elf[1], elves):
            for m in movements:
                if m == 'north' and can_move_north(elf[0],elf[1],elves):
                    newPos = (elf[0], elf[1]+1)
                    break
                elif m == 'south' and can_move_south(elf[0],elf[1],elves):
                    newPos = (elf[0], elf[1]-1)
                    break
                elif m == 'west' and can_move_west(elf[0],elf[1],elves):
                    newPos = (elf[0]-1, elf[1])
                    break
                elif m == 'east' and can_move_east(elf[0],elf[1],elves):
                    newPos = (elf[0]+1, elf[1])
                    break

        if elf != newPos:
            moved = True
        proposedMoves[elf] = newPos
        counts[newPos] = counts.get(newPos,0) + 1

    # End of round, make moves, adjust movement
    newMoves = set()
    for elf, newPos in proposedMoves.items():
        if counts[newPos] > 1:  newMoves.add(elf)
        else:                   newMoves.add(newPos)
    movements = movements[1:] + movements[:1]
    elves = newMoves

    # Solve Part A
    if rounds == 10:
        edgeLeft   = min([e[0] for e in elves])
        edgeRight  = max([e[0] for e in elves])
        edgeBottom = min([e[1] for e in elves])
        edgeTop    = max([e[1] for e in elves])
        logging.info("{} <= x <= {}, {} <= y <= {}".format(edgeLeft, edgeRight, edgeBottom, edgeTop))
        partA = (edgeRight - edgeLeft + 1) * (edgeTop - edgeBottom + 1) - len(elves)
        
    # Solve Part B
    if moved == False:
        break

logging.info("23a. Number of empty squares: {}".format(partA,))
logging.info("23b. Number of rounds: {}".format(rounds,))


2023-01-24 09:42:54 INFO: Got 71 lines from prob23input.txt.
2023-01-24 09:42:54 INFO: -6 <= x <= 74, -5 <= y <= 74
2023-01-24 09:43:00 INFO: 23a. Number of empty squares: 3970
2023-01-24 09:43:00 INFO: 23b. Number of rounds: 923


# 24.  Blizzard Basin

## 24a.  What is the fewest number of minutes required to avoid the blizzards and reach the goal?

## 24b.  What is the fewest number of minutes required to reach the goal, go back to the start, then reach the goal again?

In [46]:
# Depth First Search with pruning.
# For part II, the three legs can be solved independently.
#  Reason that if first leg has fasted time t1 and another time t2,
#  then wait at finish until t1 equals t2 before starting again is
#  a viable path to be at finish at time t2... so no need to hold
#  on to remaining states.

# 2nd attempt
filename = 'prob24input.txt'
lines = get_string_input(filename)
'''lines = ['#E######',
         '#>>.<^<#',
         '#.<..<<#',
         '#>v.><>#',
         '#<^v^^>#',
         '######.#']
'''
directions = ['right','down','left','up']

n, m = len(lines), len(lines[0])
blizzards = set()
edges = [0, n-1, 0, m-1]
for i in range(n):
    for j in range(m):
        if lines[n-i-1][j] == '>':
            blizzards.add((j,i,0))
        elif lines[n-i-1][j] == 'v':
            blizzards.add((j,i,1))
        elif lines[n-i-1][j] == '<':
            blizzards.add((j,i,2))
        elif lines[n-i-1][j] == '^':
            blizzards.add((j,i,3))

def is_east_blizzard_free(x, y, t, n, m, blizzards):
    retval = True
    newX = x - (t % (m-2))
    if newX <= 0:
        newX += (m - 2)
    if (newX, y, 0) in blizzards:
        retval = False
    return retval

def is_west_blizzard_free(x, y, t, n, m, blizzards):
    retval = True
    newX = ((x + (t % (m-2))) % (m-2))
    if newX >= m-1:
        newX -= (m - 2)
    if (newX, y, 2) in blizzards:
        retval = False
    return retval

def is_south_blizzard_free(x, y, t, n, m, blizzards):
    retval = True
    newY = y + (t % (n-2))
    if newY >= n-1:
        newY -= (n - 2)
    if (x, newY, 1) in blizzards:
        retval = False
    return retval

def is_north_blizzard_free(x, y, t, n, m, blizzards):
    retval = True
    newY = y - (t % (n-2))
    if newY <= 0:
        newY += (n - 2)
    if (x, newY, 3) in blizzards:
        retval = False
    return retval


def is_blizzard_free(x, y, t, n, m, blizzards):
    retval = is_east_blizzard_free(x, y, t, n, m, blizzards)
    if retval == True:
        retval = is_west_blizzard_free(x, y, t, n, m, blizzards)
    if retval == True:
        retval = is_south_blizzard_free(x, y, t, n, m, blizzards)
    if retval == True:
        retval = is_north_blizzard_free(x, y, t, n, m, blizzards)
    return retval

    
def make_all_moves_from_cur(curX, curY, curT, curD, n, m, blizzards, discovered):
    retval = []
        
    # wait
    if (curX, curY) == start or is_blizzard_free(curX, curY, curT+1, n, m, blizzards):
        retval.append((curT+1, curD, curX, curY))
    # moveEast
    if (curX < m-2) and (0 < curY < n-1) and is_blizzard_free(curX+1, curY, curT+1, n, m, blizzards):
        retval.append((curT+1, curD+1, curX+1, curY))
    # moveSouth
    if (curX, curY-1) == dest:
        retval.append((curT+1, curD+1, curX, curY-1))
    if (curY > 1) and (0 < curX < m-1) and is_blizzard_free(curX, curY-1, curT+1, n, m, blizzards):
        retval.append((curT+1, curD+1, curX, curY-1))
    # moveWest
    if (curX > 1) and (0 < curY < n-1) and is_blizzard_free(curX-1, curY, curT+1, n, m, blizzards):
        retval.append((curT+1, curD-1, curX-1, curY))
    # moveNorth
    if (curX, curY+1) == start:
        retval.append((curT+1, curD-1, curX, curY+1))
    if (curY < n-2) and (0 < curX < m-1) and is_blizzard_free(curX, curY+1, curT+1, n, m, blizzards):
        retval.append((curT+1, curD-1, curX, curY+1))

    return [s for s in retval if s not in discovered]



start, dest = (1, n-1), (m-2, 0)
maxDist = n + m - 4

minTime, poss, disc = None, [], set()  # best time to target, possible states, discovered states
poss.append((0,0,1,n-1)) # (time, man. dist. from start, x-pos, y-pos)
disc.add((0,0,1,n-1))
while len(poss) > 0:
    curT, curD, curX, curY = poss.pop(0)
    
    if (minTime == None) or (curT + (maxDist - curD) <= minTime):
        # state still has possibility of leading to new best path
        if (curX, curY) == dest:
            logging.info("Success! time={}, len(poss)={}, len(disc)={}".format(curT,len(poss),len(disc)))
            minTime = min([x for x in [minTime,curT] if x is not None])
        else:
            for s in make_all_moves_from_cur(curX, curY, curT, curD, n, m, blizzards, disc):
                poss.append(s)
                disc.add(s)
            poss = sorted(poss, key=lambda x: [-1*x[1],x[0]])  # sort by distance (desc) then time

firstLegTime = minTime
logging.info("Final count = {}".format(count,))
logging.info("len(possibles) = {}".format(len(poss),))
logging.info("len(discovered) = {}".format(len(disc),))
logging.info("24a. Min Time to Goal is: {}".format(firstLegTime,))

# Second Leg
minTime, poss, disc = None, [], set()  # best time to target, possible states, discovered states
poss.append((firstLegTime,0,m-2,0)) # (time, man. dist. from start, x-pos, y-pos)
disc.add((firstLegTime,0,m-2,0))
while len(poss) > 0:
    curT, curD, curX, curY = poss.pop(0)

    if (minTime == None) or (curT + (maxDist + curD) <= minTime):  # Note: curD <= 0
        # state still has possibility of leading to new best path
        if (curX, curY) == start:
            logging.info("Success! time={}, len(poss)={}, len(disc)={}".format(curT,len(poss),len(disc)))
            minTime = min([x for x in [minTime,curT] if x is not None])
        else:
            for s in make_all_moves_from_cur(curX, curY, curT, curD, n, m, blizzards, disc):
                poss.append(s)
                disc.add(s)
            poss = sorted(poss, key=lambda x: [x[1],x[0]])  # sort by distance (asc) then time

secondLegTime = minTime
logging.info("Final count = {}".format(len(disc),))
logging.info("Min Time for second leg: {}".format(secondLegTime-firstLegTime))
logging.info("Min Time end of second leg: {}".format(secondLegTime,))

# Third Leg
minTime, poss, disc = None, [], set()  # best time to target, possible states, discovered states
poss.append((secondLegTime,0,1,n-1)) # (time, man. dist. from start, x-pos, y-pos)
disc.add((secondLegTime,0,1,n-1))
while len(poss) > 0:
    curT, curD, curX, curY = poss.pop(0)
    if (minTime == None) or (curT + (maxDist - curD) <= minTime):
        # state still has possibility of leading to new best path
        if (curX, curY) == dest:
            logging.info("Success! time={}, len(poss)={}, len(disc)={}".format(curT,len(poss),len(disc)))
            minTime = min([x for x in [minTime,curT] if x is not None])
        else:
            for s in make_all_moves_from_cur(curX, curY, curT, curD, n, m, blizzards, disc):
                poss.append(s)
                disc.add(s)
            poss = sorted(poss, key=lambda x: [-1*x[1],x[0]])  # sort by distance (desc) then time
    
thirdLegTime = minTime
logging.info("Final count = {}".format(len(disc),))
logging.info("Min Time for third leg: {}".format(thirdLegTime-secondLegTime))

logging.info("24b. Min Time to Goal is: {}".format(thirdLegTime,))



2023-01-24 09:43:36 INFO: Got 22 lines from prob24input.txt.
2023-01-24 09:43:37 INFO: Success! time=504, len(poss)=538, len(disc)=1916
2023-01-24 09:43:37 INFO: Success! time=494, len(poss)=549, len(disc)=2569
2023-01-24 09:43:37 INFO: Success! time=492, len(poss)=545, len(disc)=2579
2023-01-24 09:43:37 INFO: Success! time=476, len(poss)=586, len(disc)=3942
2023-01-24 09:43:37 INFO: Success! time=464, len(poss)=580, len(disc)=5423
2023-01-24 09:43:37 INFO: Success! time=456, len(poss)=511, len(disc)=7555
2023-01-24 09:43:37 INFO: Success! time=448, len(poss)=519, len(disc)=7887
2023-01-24 09:43:40 INFO: Success! time=444, len(poss)=769, len(disc)=37851
2023-01-24 09:43:40 INFO: Success! time=397, len(poss)=799, len(disc)=43134
2023-01-24 09:43:45 INFO: Success! time=373, len(poss)=961, len(disc)=91558
2023-01-24 09:43:49 INFO: Final count = 112312725
2023-01-24 09:43:49 INFO: len(possibles) = 0
2023-01-24 09:43:49 INFO: len(discovered) = 125412
2023-01-24 09:43:49 INFO: 24a. Min Time 

# 25.  Full of Hot Air

## 25a.  What SNAFU number do you supply to Bob's console?

## 25b.  Free

In [49]:
# Looking at numbers in other bases.
# Thinking of xyz in base b as x*b^2 + y*b^1 + z*b^0 is helpful.


# 2nd attempt
filename = 'prob25input.txt'
lines = get_string_input(filename)
#lines = ['1=-0-2','12111','2=0=','21','2=01','111','20012','112','1=-1=','1-12','12','1=','122']

def read_number_in_weird_base_5(line):
    retval = 0
    valDict = {'0':0, '1':1, '2':2, '-':-1, '=':-2}
    for i, c in enumerate(reversed(line)):
        retval += valDict[c] * (5**i)
    return retval


def get_number_in_base_5(n):
    ''' Return digits as array with retval[i] == d_i of base5 integer'''
    #logging.info("n = {}".format(n,))
    retval = []
    while n > 0:
        retval.append(n % 5)
        n = n//5
    return retval


def get_number_in_weird_base_5(digits):
    ''' digits is an array with digits[i] == d_i '''
    #logging.info("reversed digits base 5 = {}".format(digits,))
    retval = []
    carry = 0
    for d in digits:
        newD = d + carry
        carry = 0
        retval.append({0:'0', 1:'1', 2:'2', 3:'=', 4:'-', 5:'0'}[newD])
        if newD > 2:
            carry = 1
    if carry == 1:
        retval.append('1')
    return ''.join(reversed(retval))  


listSum = 0
for l in lines:
    listSum += read_number_in_weird_base_5(l)

base5listSum = get_number_in_base_5(listSum)
weirdBase5num = get_number_in_weird_base_5(base5listSum)
logging.info("{} = {} base 5, weird 5 = '{}'".format(listSum, base5listSum, weirdBase5num))

logging.info("25a. Weird base 5 num '{}'".format(weirdBase5num,))


2023-01-24 09:45:08 INFO: Got 110 lines from prob25input.txt.
2023-01-24 09:45:08 INFO: 33841257499180 = [0, 1, 2, 3, 3, 4, 4, 0, 4, 3, 4, 3, 3, 2, 4, 3, 1, 4, 3, 1] base 5, weird 5 = '2--2-0=--0--100-=210'
2023-01-24 09:45:08 INFO: 25a. Weird base 5 num '2--2-0=--0--100-=210'
